# 0. Data gathering
All data will be aggreagte to daily for furhter aggregation and feature engineering

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#folders
data_folder = "data"
#data_folder = os.path.join("D:","bthe_downloads")

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

In [ ]:
#only used for initial data downliading and parsing

try:
    from unittest import result
    import cdsapi #additional file needed to run. See docu
    import requests
    import xarray as xr
except:
    print("libs import failed. Not needed, unless th era5 data is to be downloaded anew and recompile the .nc to .csv")

In [ ]:
#general parsing and aggregating of files
save_data                   = False

#chapter 0.1
t2_run_era5_download        = False
t2m_compile_df              = False

#chapter 0.2
soi_run_era5_download       = False
soi_compile_df              = False

#chapter 0.5
pv_run_era5_download        = False
pv_compile_df               = False

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#custom and dynamic aggregation funciton

def dynamic_aggregation(df, grouping_col):
    """change to daily intervall"""

    #get mean and std
    df_mean = df.groupby([grouping_col], as_index = True).mean()
    df_std = df.groupby([grouping_col], as_index = True).std()

    #combine
    df = df_mean.join(other = df_std, lsuffix="_mean", rsuffix='_std')
    df = df.round(2)

    return df

## 0.1 General weather and temperature data (t2m)
source: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

In [ ]:
#define borders
north   = 47.8
east    = 10.5
south   = 45.8
west    = 6.0

lons = [west, west, east, east, west]
lats = [south, north, north, south, south]

#lons = [6,6,10.5,10.5,6]
#lats = [45.8,47.8,47.8,45.8,45.8]

#create plot
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lons,
    lat = lats,
    marker = {'size': 10})
)

#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 8.4, 'lat': 46.85},
        'style': "carto-positron",
        'zoom': 5})


scale_show(fig)

In [ ]:
#data source: https://cds.climate.copernicus.eu/cdsapp#!/home

class Wrapper():

    #class variables
    folder_name : str = None

    #functionality
    def main(all_vars : bool, data_folder):

        Wrapper.folder_name = os.path.join(data_folder,"raw_t2m")

        start_year : int        = 1979 #1979
        end_year : int          = 2023 #2023

        years : list            = Wrapper.generate_year_list(start = start_year, end = end_year)
        variables : list        = Wrapper.generate_var_list(all = all_vars)

        #main loop for downloading data
        Wrapper.log(f"Downloading startet for range: {start_year} - {end_year}")

        for year in years:

            print(f"Processing {year}")
            result : str = Wrapper.request(year, variables, all_vars)

        #tranforms and saves data as a csv for later processing in pandas
        Wrapper.generate_df()

        return

    def generate_year_list(start:int, end:int):

        year_list_str : list = [str(year) for year in range(start,end)]
        return year_list_str

    def generate_var_list( all : bool):

        if all == True:
            return [
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
                'clear_sky_direct_solar_radiation_at_surface',
                'surface_pressure',]
        else:
            return ['2m_temperature']

    def generate_df():

        downloads = Wrapper.folder_name
        files = os.listdir(downloads)
        files = [file for file in files if file[-3:] == ".nc"]

        for file in files:

            #open .nc files
            file = os.path.join(downloads,file)

            ds = xr.open_dataset(file)
            df = ds.to_dataframe()

            #save df
            file = os.path.basename(file)
            name = f"{file[:-3]}.csv"
            path = os.path.join(Wrapper.folder_name,name)
            df.to_csv(path)

    def download_data(result:str, year:str, all_vars:bool):
        """[deprecated]"""

        #genearte download and saving path
        path : str          = Wrapper.download_path()
        file_name : str     = f"t2m_{year}_allvars_{all_vars}.nc" #type nasCat data
        file_path :str      = os.path.join(path,file_name)
        print(file_path)

        #get download link
        try:
            link_start : int    = result.index("location=") + len ("location=")
            url : str           = result[link_start:-1]
        except:
            Wrapper.log(f"{year}: The api response does not contain a  download link")
            return

        #retrieve data from web page and save it
        try:
            response = requests.get(url)
        except:
            Wrapper.log(f"{year}: The download url is not valid")
            return

        open(file_path, "wb").write(response.content)

        return

    def download_path():

        folder_name = Wrapper.folder_name

        if os.path.isdir(folder_name) == False:
            os.makedirs(folder_name)

        download_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), folder_name)

        return download_path

    def file_path(year, all_vars):

        #genearte download and saving path
        path : str          = Wrapper.folder_name
        file_name : str     = f"t2m_{year}_allvars_{all_vars}.nc" #type nasCat data
        file_path :str      = os.path.join(path,file_name)

        print(file_path)
        return file_path

    def log(message : str):

        #create log entry
        log_time : str = datetime.now()
        message = f"{log_time},{message}\n"

        #write log entry
        file_object = open('era5_log.txt', 'a')
        file_object.write(message)
        file_object.close()

        return

    def request(year:list, variable:list, all_vars:bool):
        # see: https://www.latlong.net/

        c = cdsapi.Client()
        file_path = Wrapper.file_path(year = year, all_vars = all_vars)

        request = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': variable,
                'year': year,
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'area': [47.8, 6, 45.8,10.5,],
                'format': 'netcdf',
            },
            file_path
        )

        return str(request)

if t2_run_era5_download is True:
    Wrapper.main(all_vars = True, data_folder = data_folder)

In [ ]:
#aggregates to weekly interval data

class Agg_t2m():

    data = None
    t2m_data :str = None

    def main(data_folder):

        Agg_t2m.data = data_folder
        Agg_t2m.t2m_data = os.path.join(data_folder,"raw_t2m")

        files : list        = os.listdir(Agg_t2m.t2m_data)
        csv_files : list    = [file for file in files if file[-4:] == ".csv"]

        dfs : list          = []

        for csv in csv_files:

            print(f"Processing: {csv}", end = "\r")
            df = Agg_t2m.lonlat_mean(csv = csv)
            dfs.append(df)

        df = Agg_t2m.merge(dfs)
        df = Agg_t2m.aggregate(df)
        Agg_t2m.save(df, data_folder)

        del dfs, df
        return

    def lonlat_mean(csv : str):

        #average over lon and lattitude
        df = pd.read_csv(os.path.join(Agg_t2m.t2m_data, csv))
        df = df.drop(labels = ["longitude","latitude"], axis = 1)
        df_avg_lon_lat = df.groupby("time").mean()

        del df #free up memory
        return df_avg_lon_lat

    def aggregate(df):

        #generate gorup index as date
        df.reset_index(drop = False, inplace = True)
        df["time"] = pd.to_datetime(df["time"])
        df["date"] = df["time"].dt.date

        #set date as index
        df.set_index("date", drop = True, inplace = True)

        #drop time column
        df.drop(labels = ["time"], inplace = True, axis = 1)

        #aggreagte data
        df = df.groupby(["date"], as_index = True).mean()

        return df

    def merge(dfs):

        df = pd.concat(dfs)
        #df.set_index(keys = "date", inplace = True)
        df.sort_index(inplace = True)

        return df

    def save(df, data_folder):

        df.to_csv(os.path.join(data_folder, "df_t2m.csv"))
        return

if t2m_compile_df is True:
    Agg_t2m.main(data_folder)

In [ ]:
df_t2m = pd.read_csv(os.path.join(data_folder, "df_t2m.csv"), index_col = "date")
df_t2m.head()

In [ ]:
df_t2m.tail()

## 0.2 Southern oscilation index (SOI)
source:
- http://www.bom.gov.au/climate/mjo/
- https://www.climate.gov/news-features/understanding-climate/climate-variability-southern-oscillation-index

formula:
- http://www.bom.gov.au/climate/glossary/soi.shtml
- https://www.ncei.noaa.gov/access/monitoring/enso/soi


The Southern Oscillation Index (SOI) is calculated using the atmospheric pressure difference between Tahiti and Darwin, Australia. The most common approach is to use monthly mean sea level pressure values for these two locations, which are then standardized and combined to create the SOI.

- Tahiti: 17.5░S / 149.5░W
- Darwin,  12.5░S / 131.5░E

or

- Tahiti (https://www.latlong.net/place/papeete-french-polynesia-30701.html):
    - lat = -17.53
    - lon = -149.56
- Darwin (https://www.latlong.net/place/darwin-northern-territory-australia-5517.html):
    - lat = -12.46
    - lon = 130.84

Here is a source for more information on the calculation of the SOI:
https://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/soi.html


In [ ]:
lonlat_data = {
    "place"     : ["tahiti",    "darwin"],
    "lat"       : [-17.53,        -12.46],
    "lon"       : [-149.56,       130.84],
    "size"      : [1, 1],
}

df = pd.DataFrame(lonlat_data)

fig = px.scatter_mapbox(
    data_frame= df,
    lat="lat",
    lon="lon",
    size = "size",
    hover_name = "place",
    size_max = 15,
    color_continuous_scale = plt_style_s,
)


#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 178, 'lat':-18},
        'style': "carto-positron",
        'zoom': 2})

scale_show(fig)

In [ ]:
#lat = north / south
#lon = east / west
df

In [ ]:
#fetch the sea surface preassure data from copernicus

class Wrapper():

    #class variables
    folder_name : str = None

    #functionality
    def main(data_folder):

        Wrapper.folder_name = os.path.join(data_folder,"raw_soi")

        start_year : int        = 1979 #1979
        end_year : int          = 2023 #2024

        years : list            = Wrapper.generate_year_list(start = start_year, end = end_year)
        variables : list        = Wrapper.generate_var_list()

        #set the two cites as previuously defined
        cities = {
            "tahiti" : [-17.50, -149.60, -17.50, -149.55,],
            "darwin" : [-12.45, 130.80, -12.50, 130.85,],
            }

        #main loop for downloading data
        Wrapper.log(f"Downloading startet for range: {start_year} - {end_year}")

        for year in years:
            for key in cities:
                print(f"Processing {year}, {key}")
                result : str = Wrapper.request(year = year, variable = variables, city = key , area = cities[key])

        #tranforms and saves data as a csv for later processing in pandas
        Wrapper.generate_df()

        return

    def generate_year_list(start:int, end:int):

        year_list_str : list = [str(year) for year in range(start,end)]
        return year_list_str

    def generate_var_list():

        return ["mean_sea_level_pressure"]

    def generate_df():

        downloads = Wrapper.folder_name
        files = os.listdir(downloads)
        files = [file for file in files if file[-3:] == ".nc"]

        for file in files:

            #open .nc files
            file = os.path.join(downloads,file)
            ds = xr.open_dataset(file)
            df = ds.to_dataframe()

            #save df
            file = os.path.basename(file)
            name = f"{file[:-3]}.csv"
            path = os.path.join(Wrapper.folder_name,name)
            df.to_csv(path)

    def download_path():

        folder_name = Wrapper.folder_name

        if os.path.isdir(folder_name) == False:
            os.makedirs(folder_name)

        download_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), folder_name)

        return download_path

    def file_path(year, city):

        #genearte download and saving path
        path : str          = Wrapper.folder_name
        file_name : str     = f"soi_{year}_{city}.nc" #type nasCat data
        file_path :str      = os.path.join(path,file_name)

        return file_path

    def log(message : str):

        #create log entry
        log_time : str = datetime.now()
        message = f"{log_time},{message}\n"

        #write log entry
        file_object = open('era5_log.txt', 'a')
        file_object.write(message)
        file_object.close()

        return

    def request(year:list, variable:list, area : list, city : str):
        # see: https://www.latlong.net/

        c = cdsapi.Client()
        file_path = Wrapper.file_path(year = year, city = city)

        request = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': variable,
                'area': area,
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year,
            },
            file_path
        )

        return str(request)

if soi_run_era5_download is True:
    Wrapper.main(data_folder = data_folder)

In [ ]:
class Compile_soi():

    data = None
    t2m_data :str = None

    def main(data_folder):

        Compile_soi.data = data_folder
        Compile_soi.t2m_data = os.path.join(data_folder,"raw_soi")

        files : list        = os.listdir(Compile_soi.t2m_data)

        csv_files_tahiti : list    = [file for file in files if file[-4:] == ".csv" and "tahiti" in file]
        csv_files_darwin : list    = [file for file in files if file[-4:] == ".csv" and "darwin" in file]

        dfs : list          = []

        for csv_tahiti, csv_darwin in zip(csv_files_tahiti, csv_files_darwin):
            print(f"processing:{csv_tahiti}, {csv_darwin}")

            #remove lon lat dependency
            df_tahiti = Compile_soi.lonlat_mean(csv = csv_tahiti)
            df_darwin = Compile_soi.lonlat_mean(csv = csv_darwin)

            #combine the dataframes
            df = Compile_soi.combine(df_tahiti = df_tahiti, df_darwin = df_darwin) #also aggreagte the data to daily interval
            del df_tahiti, df_darwin

            dfs.append(df)

        df = Compile_soi.merge(dfs)
        df = Compile_soi.calculate_soi(df)
        Compile_soi.save(df, Compile_soi.data)

        return

    def lonlat_mean(csv : str):

        #average over lon and lattitude
        df = pd.read_csv(os.path.join(Compile_soi.t2m_data, csv))
        df = df.drop(labels = ["longitude","latitude"], axis = 1)
        df_avg_lon_lat = df.groupby("time").mean()

        del df #free up memory
        return df_avg_lon_lat

    def combine(df_tahiti, df_darwin):

        #combine into one data frame
        df = df_tahiti.join(other = df_darwin, lsuffix = "_darwin", rsuffix = "_tahiti", on = "time")

        #aggregate
        df = Compile_soi.aggregate(df)

        return df

    def aggregate(df):

        #generate gorup index as date
        df.reset_index(drop = False, inplace = True)
        df["time"] = pd.to_datetime(df["time"])
        df["date"] = df["time"].dt.date

        #set date as index
        df.set_index("date", drop = True, inplace = True)

        #drop time column
        df.drop(labels = ["time"], inplace = True, axis = 1)

        #aggreagte data
        df = df.groupby(["date"], as_index = True).mean()

        return df

    def merge(dfs):

        df = pd.concat(dfs)
        df.sort_index(inplace=True)

        return df

    def calculate_soi(df):
        """sorce of formula: http://www.bom.gov.au/climate/glossary/soi.shtml"""
        # soi = 10 * (p_diff - p_diff_mean) / (p_diff_std)

        #boilerplate
        df["day_index"] = df.index
        df["day_index"] = pd.to_datetime(df.index, errors='coerce')
        df["day_index"] = df["day_index"].dt.strftime('%d-%m')

        #get pressure diff
        df["p_diff"] = df["msl_tahiti"] - df["msl_darwin"]

        #get long term values
        df_long_term = df.groupby(["day_index"], as_index = False).agg(
            p_diff_mean     = ("p_diff", "mean"),
            p_diff_std      = ("p_diff", "std"),
        )

        #delete this later
        df_long_term.fillna(20, inplace = True)

        #join data
        df["day_index"] = df["day_index"].astype("string"); df_long_term["day_index"] = df_long_term["day_index"].astype("string")
        df.reset_index(inplace = True)
        df = pd.merge(df, df_long_term, on = "day_index", how = "left")

        #calculate soi
        df["soi"] = (df["p_diff"] - df["p_diff_mean"]) / df["p_diff_std"]
        df["soi"] = df["soi"] * (-1) #inverse

        #clean up
        df.drop(labels = ["day_index", "p_diff", "msl_tahiti", "msl_darwin", "p_diff_mean", "p_diff_std"], axis = 1, inplace = True)

        #set index
        df.set_index("date", inplace = True, drop = True)

        return df

    def save(df, data_folder):

        df.to_csv(os.path.join(data_folder, "df_soi.csv"))
        return

if soi_compile_df is True:
    Compile_soi.main(data_folder = data_folder)


In [ ]:
df_soi = pd.read_csv(os.path.join(data_folder, "df_soi.csv"))
df_soi.head(5)

In [ ]:
#complicate, but the normal way throws an unsolveable error
df_soi["year"] = pd.DatetimeIndex(df_soi["date"]).year
df_soi["month"] = pd.DatetimeIndex(df_soi["date"]).month

#get mean
df_soi = df_soi.groupby(["year", "month"], as_index = False).mean()
df_soi["day"] = "01"
df_soi["index"] = pd.to_datetime(df_soi[["year", "month", "day"]])

df_soi.shape

In [ ]:
#compare to monthly values of soi by trusted source
#source: https://www.cpc.ncep.noaa.gov/data/indices/soi

#read compare file
df_enso_raw = pd.read_csv(os.path.join(data_folder, "raw_enso", "raw_enso.csv"))


#new columns
enso_dict : dict = {
    "index" : [],
    "year" : [],
    "month" : [],
    "enso" : [],
}

#iterrate over df to retrieve values
for year in df_enso_raw["year"].to_list():
    for month in df_enso_raw.columns.to_list()[1:]:

        enso : float        = float(df_enso_raw.loc[df_enso_raw["year"] == year][str(month)])
        index : str =       f"{year}-{month}"

        enso_dict["index"].append(index)
        enso_dict["year"].append(int(year))
        enso_dict["month"].append(int(month))
        enso_dict["enso"].append(enso)

#create new df
df_enso = pd.DataFrame(data = enso_dict)
df_enso.set_index("index", drop = True, inplace = True)

#set column
df_enso["day"] = "01"
df_enso["index"] = pd.to_datetime(df_enso[["year", "month", "day"]])

df_enso.shape


In [ ]:
#create comparison
df_soi_comp = pd.DataFrame()
df_soi_comp["index"] = df_soi["index"]

df_soi_comp["soi_calculated"] = df_soi["soi"].tolist()
df_soi_comp["soi_source"] = df_enso["enso"].tolist()

In [ ]:
#plot for visual check
#finding: they seem to be scaled differntly, but the shape is the same
fig = px.line(
    data_frame = df_soi_comp,
    x = "index",
    y = ["soi_calculated", "soi_source"],
    title = "SOI comparison",
    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:

df_soi_comp["sacling"]  = df_soi_comp["soi_source"].abs() / df_soi_comp["soi_calculated"].abs()
scale_factor : float = df_soi_comp["sacling"].mean()

df_soi_comp["soi_source"] = df_soi_comp["soi_source"] / scale_factor

In [ ]:
fig = px.line(
    data_frame = df_soi_comp,
    x = "index",
    y = ["soi_calculated", "soi_source"],
    title = "SOI comparison scaled",
    color_discrete_sequence = plt_style_s
)

scale_show(fig)


## 0.3 Madden julien oscillation (MJO)
- Source: http://www.bom.gov.au/climate/mjo/
- Note:
    - The data was downloaded and prepared manually as a .csv file (reformating).
    - The datapoint from the year 1978 are missing. A broken up date time series does not make sense. The enso also starts from the year 1979. Therefore the data from year < 1979 is being dropped
    - The aggreagtion will be done in two ways:
        - Values of the last day of month
        - Aggregation:
            - RMM1, RMM2, apmlitude: std, mean
            - phase: mode
- Range: 1979 - 2023

In [ ]:
df_mjo_raw = pd.read_csv(os.path.join(data_folder,"raw_mjo","raw_mjo.csv"))
df_mjo_raw.head()

In [ ]:
#understanding the mjo data
fig = px.line(
    data_frame = df_mjo_raw.loc[310:390],
    x = "RMM1",
    y = "RMM2",
    color = "month",

    title = "MJO",
    color_discrete_sequence = plt_style_s,
    width = 500,
    height = 500,

    range_x = (3,-3),
    range_y = (3,-3),
)

fig.show()

In [ ]:
fig = px.line(
    data_frame = df_mjo_raw,
    y = "amplitude",

    title = "MJO",
    color_discrete_sequence = plt_style_s,
    **size,
)

scale_show(fig)

In [ ]:
#drop unneeded cols
df_mjo_raw.drop(axis = 1, labels = ["Unnamed: 0","unnamed", "MissingValue=1.E36or999"], inplace = True)
df_mjo = df_mjo_raw.loc[df_mjo_raw["year"] >= 1979]
df_mjo.head()

In [ ]:
fig = px.line(
    data_frame = df_mjo,
    y = "amplitude",

    title = "MJO",
    color_discrete_sequence = plt_style_s,
    **size,
)

scale_show(fig)

In [ ]:
df_mjo.describe()

In [ ]:
df_mjo.isna().sum()

In [ ]:
#clean up and prep data
df_mjo.reset_index(inplace = True, drop = True)
df_mjo['date'] = pd.to_datetime(df_mjo[["year", "month", "day"]])
df_mjo.drop(labels = ["year", "month", "day"], axis = 1, inplace = True)

#set index
df_mjo.set_index("date", drop = True, inplace = True)

#clean up cols
df_mjo = df_mjo.add_prefix(prefix = "mjo_")
df_mjo.columns = [x.lower() for x in df_mjo.columns]

#sort
df_mjo.sort_index(inplace = True)

df_mjo.head()

In [ ]:
if save_data is True:
    df_mjo.to_csv(os.path.join(data_folder, "df_mjo.csv"))

## 0.4 Arctic oscillation index (AO)
- source: https://ftp.cpc.ncep.noaa.gov/cwlinks/

In [ ]:
df_ao_raw = pd.read_csv(os.path.join(data_folder, "raw_ao", "norm_daily_ao_cda_z1000_19500101_current.csv"))
df_ao_raw.head(10)

In [ ]:
fig = px.line(
    data_frame = df_ao_raw,
    y = "ao_index_cdas",

    title = "AO index",
    **size,
    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
df_ao = df_ao_raw.rename(mapper = {"ao_index_cdas":"ao"}, inplace = False, axis = 1)
df_ao.describe()

In [ ]:
df_ao.isna().sum()

In [ ]:
df_ao[df_ao.isna().any(axis=1)]

In [ ]:
fig = px.line(
    data_frame = df_ao.loc[(df_ao["year"] == 2003) & (df_ao["month"].isin([4,5]))],
    y = "ao",
    title = "AO 2003",
    **size,
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
#use interpolation to fill na value
df_ao.interpolate(method = "linear", inplace = True)

In [ ]:
fig = px.line(
    data_frame = df_ao.loc[(df_ao["year"] == 2003) & (df_ao["month"].isin([4,5]))],
    y = "ao",
    title = "ao 2003",
    **size,
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
df_ao.isna().sum()

In [ ]:
df_ao.head(5)

In [ ]:
#clean up and prep data
df_ao.reset_index(inplace = True, drop = True)
df_ao["date"] = pd.to_datetime(df_ao[["year", "month", "day"]])
df_ao.drop(labels = ["year", "month", "day"], axis = 1, inplace = True)

#set index
df_ao.set_index("date", drop = True, inplace = True)

#aggregagation not needed

#sort
df_ao.sort_index(inplace = True)

df_ao.head()

In [ ]:
if save_data is True:
    df_ao.to_csv(os.path.join(data_folder, "df_ao.csv"))

## 0.5 North atlantic oscilation (NAO)
source: https://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/nao.shtml

In [ ]:
df_nao_raw = pd.read_csv(os.path.join(data_folder, "raw_nao","norm_daily_nao_cdas_z500_19500101_current.csv"))
df_nao_raw.head()

In [ ]:
fig = px.line(
    data_frame = df_nao_raw,
    y = "nao_index_cdas",
    title = "NAO",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
df_nao_raw.isna().sum()

In [ ]:
df_nao_raw[df_nao_raw.isna().any(axis=1)]

In [ ]:
#fill na with interpolation
df_nao = df_nao_raw.interpolate(method = "linear")
df_nao.isna().sum()

In [ ]:
#clean up and prep data
df_nao.reset_index(inplace = True, drop = True)
df_nao["date"] = pd.to_datetime(df_nao[["year", "month", "day"]])
df_nao.drop(labels = ["year", "month", "day"], axis = 1, inplace = True)

#set index
df_nao.set_index("date", drop = True, inplace = True)

#aggregagation not needed

#rename col
df_nao.rename(mapper = {"nao_index_cdas" : "nao"}, axis = 1, inplace = True)

#sort
df_nao.sort_index(inplace = True)

df_nao.head()

In [ ]:
if save_data is True:
    df_nao.to_csv(os.path.join(data_folder, "df_nao.csv"))

## 0.6 Polar vortex data (PV)

relevant preassure level: 1000 hPa level (see bthe, chapter 2.4 on polar vortex and ao)

In [ ]:
#define borders
north   = 90
south   = 45.0
west    = 8.0
east    = 8.0


lons = [west, west, east, east, west]
lats = [south, north, north, south, south]

#lons = [6,6,10.5,10.5,6]
#lats = [45.8,47.8,47.8,45.8,45.8]

#create plot
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lons,
    lat = lats,
    marker = {'size': 10})
)

#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 8.4, 'lat': 60},
        'style': "carto-positron",
        'zoom': 3})


scale_show(fig)

In [ ]:
#resolution of data
#define borders

lons = []
lats = []

for lat in range(round(south) , round(north)):
    for lon in [8,9]:
        lons.append(lon)
        lats.append(lat)

#lons = [6,6,10.5,10.5,6]
#lats = [45.8,47.8,47.8,45.8,45.8]

print(f"datapoint per time step: {len(lons)}")
print(f"image resolution: {len(set(lons))} x {len(set(lats))}")

#create plot
fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = lons,
    lat = lats,
    marker = {'size': 10})
)

#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 8.4, 'lat': 60},
        'style': "carto-positron",
        'zoom': 3})


scale_show(fig)

In [ ]:
#data source: https://cds.climate.copernicus.eu/cdsapp#!/home

class Wrapper():

    #class variables
    folder_name : str = None

    #functionality
    def main(data_folder):

        Wrapper.folder_name = os.path.join(data_folder,"raw_pv")

        start_year : int        = 1979 #1979
        end_year : int          = 2023 #2023

        years : list            = Wrapper.generate_year_list(start = start_year, end = end_year)
        variable : list        = Wrapper.generate_var_list()

        #main loop for downloading data
        Wrapper.log(f"Downloading startet for range: {start_year} - {end_year}")

        for year in years:
            for month in range(1,13):

                print(f"Processing {year}-{month}")

                try:
                    emailer.message(f"Downloading pv: {year}-{month}")
                except:
                    pass

                result : str = Wrapper.request(year = year, month = month ,variable = variable)

        #tranforms and saves data as a csv for later processing in pandas
        Wrapper.generate_df()

        return

    def generate_year_list(start:int, end:int):

        year_list_str : list = [str(year) for year in range(start,end)]
        return year_list_str

    def generate_var_list():

        return ['u_component_of_wind', 'v_component_of_wind','temperature']

    def generate_df():

        downloads = Wrapper.folder_name
        files = os.listdir(downloads)

        for file in files:

            #open .nc files
            file = os.path.join(downloads,file)
            ds = xr.open_dataset(file)
            df = ds.to_dataframe()

            #save df
            file = os.path.basename(file)
            name = f"{file[:-3]}.csv"
            path = os.path.join(Wrapper.folder_name,name)
            df.to_csv(path)

    def download_path():

        folder_name = Wrapper.folder_name

        if os.path.isdir(folder_name) == False:
            os.makedirs(folder_name)

        download_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), folder_name)

        return download_path

    def file_path(year, month):

        #genearte download and saving path
        path : str          = Wrapper.folder_name
        file_name : str     = f"pv_{year}-{month}.nc" #type nasCat data
        file_path :str      = os.path.join(path,file_name)

        print(file_path)
        return file_path

    def log(message : str):

        #create log entry
        log_time : str = datetime.now()
        message = f"{log_time},{message}\n"

        #write log entry
        file_object = open('era5_log.txt', 'a')
        file_object.write(message)
        file_object.close()

        return

    def request(year:int, month : int, variable:list):
        # see: https://www.latlong.net/

        c = cdsapi.Client()
        file_path = Wrapper.file_path(year = year, month = month)

        print(year)

        request = c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'pressure_level': ['10', '20', '30' ,'50', '70', '100',], #now it should be correct
                'variable': variable,
                'area': [
                    90, 8, 45,
                    9,
                ],
                'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'month': month,
                'year': year,
            },
            file_path
        )

        return str(request)

if pv_run_era5_download is True:
    Wrapper.main(data_folder = data_folder)

In [ ]:
class Agg_pv():

    data = None
    pv_data :str = None

    def main(data_folder):

        Agg_pv.data = data_folder
        Agg_pv.pv_data = os.path.join(data_folder,"raw_pv")

        files : list        = os.listdir(Agg_pv.pv_data)
        csv_files : list    = [file for file in files if file[-4:] == ".csv"]

        dfs : list          = []

        for csv in csv_files:

            print(f"Processing: {csv}", end = "\r")
            df = pd.read_csv(os.path.join(Agg_pv.pv_data, csv))
            df = Agg_pv.aggregate(df)
            dfs.append(df)

        df = Agg_pv.merge(dfs)

        #add base infos
        df = Agg_pv.calculate_wind_speeds(df)
        df = Agg_pv.calculate_wind_direction(df)

        Agg_pv.save(df, data_folder)

        del dfs, df
        return

    def aggregate(df):

        #generate gorup index as date
        df.reset_index(drop = False, inplace = True)
        df["time"] = pd.to_datetime(df["time"])
        df["date"] = df["time"].dt.date

        #set date as index
        df.set_index("date", drop = True, inplace = True)

        #drop time column
        df.drop(labels = ["time"], inplace = True, axis = 1)

        #define grid resolution
        df["longitude"] = df["longitude"].round()
        df["latitude"] = df["latitude"].round()

        #aggreagte data
        df = df.groupby(["date", "longitude","latitude", "level"], as_index = True).mean()

        return df

    def merge(dfs):

        df = pd.concat(dfs)
        #df.sort_values(by = "date", inplace = True)

        return df

    def calculate_wind_speeds(df):

        df['speed'] = np.sqrt(df['u']**2 + df['v']**2)
        return df

    def calculate_wind_direction(df):

        df['direction'] = np.rad2deg(np.arctan2(df['u'], df['v'])) % 360
        df['direction'] = (df['direction'] + 90) % 360

        return df

    def save(df, data_folder):

        df.rename(mapper = {
            "longitude" : "lon",
            "latitude" : "lat",
            "u" : "wind_u",
            "v" : "wind_v",
            "t" : "temp",
        })

        if "index" in df.index.to_list():
            df.drop(labels = "index", axis = 1, inplace = True)

        df.to_csv(os.path.join(data_folder, "df_pv.csv"))
        return

if pv_compile_df is True:
    Agg_pv.main(data_folder)

In [ ]:
df_pv = pd.read_csv(os.path.join(data_folder, "df_pv.csv"), index_col = "date")
df_pv.head(10)

In [ ]:
del df_pv

## 0.7 Merge all (execpt custom polar vortex, date base)

In [ ]:
save_data = True

In [ ]:
folder_contents = os.listdir(data_folder)
csv_files = ["df_ao.csv", "df_nao.csv", "df_mjo.csv", "df_soi.csv", "df_t2m.csv"]

dfs : list = []
print(csv_files)

#read files
for csv in csv_files:

    csv_path = os.path.join(data_folder, csv)
    df = pd.read_csv(csv_path, index_col = "date")
    print(df.columns)

    dfs.append(df)

#merge files
df = dfs[0].join(other = dfs[1:])

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#reset index
df.reset_index(drop = False, inplace = True)

#presumed cleaned data range
start = "1979-01-01"
end = "2022-12-31"

#set
df = df.loc[df["date"].between(start, end)]
df.set_index("date", drop = True, inplace = True)

#clean up
[df.drop(labels = [col], inplace = True, axis = 1) for col in df.columns.tolist() if col == "index"]

#chech
df.isna().sum()

In [ ]:
#addint date time values to df for plotting
df["date_temp"] = pd.to_datetime(df.index)

#set values
df["year"] = df["date_temp"].dt.year
df["month"] = df["date_temp"].dt.month
df["day"] = df["date_temp"].dt.day

df.drop(labels = "date_temp", inplace = True, axis = 1)

df.head()

In [ ]:
df.tail()

In [ ]:
#save master data frame
if save_data is True:
    df.to_csv(os.path.join(data_folder, "df.csv"))

# 1. Data exploration

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#folders
data_folder = "data"

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1.0 General

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df.csv"))
#df_pv = pd.read_csv(os.path.join(data_folder, "df_pv.csv"))

In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
#df_pv.head()

In [ ]:
df.describe()

In [ ]:
#df_pv.describe()

In [ ]:
df.isna().sum()

In [ ]:
#df_pv.isna().sum()

In [ ]:
# Correlation
df_corr = df.corr().round(1)

# Mask to matrix
mask = np.zeros_like(df_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Viz
df_corr_viz = df_corr.mask(mask).dropna(how='all').dropna('columns', how='all')

fig = px.imshow(

    df_corr_viz,
    text_auto=True,
    color_continuous_scale = plt_style_c,

    title = "Correlation matrix",
    width = 700,
    height = 700,
    )

fig.show()

## 1.1 Temperature, pressure, wind speeds

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "u10",

    title = "Wind speed: u10",
    labels = {"u10" :"u10 [m/s]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "v10",

    title = "Wind speed: v10",
    labels = {"v10" :"v10 [m/s]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = ["v10", "u10"],
    histnorm = "probability density",
    title = "Distribution: v10",

    barmode = "overlay",
    opacity = 0.9,

    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "t2m",

    title = "Temperature",
    labels = {"t2m" :"t2m [░k]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "year",
    y = "t2m",

    title = "Temperature",
    labels = {"t2m" :"t2m [░k]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df.iloc[::10],
    x = "date",
    y = "t2m",

    title = "Temperature",
    labels = {"t2m" :"t2m [k]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df.iloc[::2],
    x = "date",
    y = "t2m",

    title = "Temperature",
    labels = {"t2m" :"t2m [k]"},

    color_discrete_sequence = plt_style_c,
    trendline = "ols",
    trendline_color_override = "red",

)

scale_show(fig)

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "t2m",
    histnorm = "probability density",
    title = "Distribution: t2m",

    color = "month",
    barmode = "stack",
    opacity = 1,

    nbins = 200,

    labels = {"t2m" : "t2m [░k]"},

    color_discrete_sequence = plt_style_c,
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df.iloc[15000:],
    x = "date",
    y = "cdir",

    title = "Clear sky solar iradiation",
    labels = {"cdir" :"cdir [j/m^2]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "cdir",

    title = "Clear sky solar iradiation",
    labels = {"cdir" :"cdir [j/m^2]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df.iloc[15000:],
    x = "date",
    y = "sp",

    title = "Surface pressure",
    labels = {"sp" :"sp [hpa]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "sp",

    title = "Surface pressure",
    labels = {"sp" :"sp [hpa]"},

    color_discrete_sequence = plt_style_s
)

scale_show(fig)

## 1.2 ENSO / SOI

In [ ]:
#some code

fig = px.line(
    data_frame = df.loc[15000:],
    x = "date",
    y = "soi",

    title = "SOI",
    color_discrete_sequence = plt_style_s,
)

fig.add_hline(
    y = 0,
    line_color="red",
)

scale_show(fig)

## 1.3 MJO

In [ ]:
#some code
#mjo_rmm1, mjo_rmm2, mjo_phase, mjo_amplitude

fig = px.line(
    data_frame = df.loc[::2],
    x = "date",
    y = "mjo_amplitude",

    title = "MJO amplitude",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
#some code
#mjo_rmm1, mjo_rmm2, mjo_phase, mjo_amplitude

fig = px.line(
    data_frame = df.loc[15000:],
    x = "date",
    y = "mjo_amplitude",

    title = "MJO amplitude",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df.iloc[300:5000],
    x = "mjo_rmm1",
    y = "mjo_rmm2",
    color = "mjo_phase",

    title = "MJO with pahse",
    color_discrete_sequence = plt_style_s,

    width = 700,
    height = 700,

    range_x = [-3,3],
    range_y = [-3,3],
)

fig.show()

In [ ]:
fig = px.line(
    data_frame = df.iloc[1000:1100],
    x = "mjo_rmm1",
    y = "mjo_rmm2",

    title = "MJO with pahse",
    color_discrete_sequence = plt_style_s,

    width = 700,
    height = 700,

    range_x = [-3,3],
    range_y = [-3,3],
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "mjo_amplitude",

    histfunc = "count",
    histnorm = "probability",

    title = "MJO amplitude distribution",
    color_discrete_sequence = plt_style_c,
    barmode = "stack",
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "mjo_amplitude",
    title = "MJO amplitude distribution",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

## 1.4 AO

In [ ]:
#some code
fig = px.line(
    data_frame = df.iloc[14000:],
    y = "ao",
    x = "date",

    title = "AO",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "ao",

    histfunc = "count",
    histnorm = "probability",

    title = "AO distribution",
    color_discrete_sequence = plt_style_c,
    barmode = "stack",
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "ao",
    title = "AO distribution",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

## 1.5 NAO

In [ ]:
#some code
fig = px.line(
    data_frame = df.iloc[14000:],
    y = "nao",
    x = "date",

    title = "NAO",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "nao",

    histfunc = "count",
    histnorm = "probability",

    title = "NAO distribution",
    color_discrete_sequence = plt_style_c,
    barmode = "stack",
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df,
    x = "month",
    y = "nao",
    title = "NAO  distribution",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

## 1.6 Variouse

In [ ]:
#some code
fig = px.line(
    data_frame = df.iloc[14000:],
    y = [df["ao"].iloc[14000:], df["mjo_amplitude"].iloc[14000:], df["soi"].iloc[14000:]],
    x = "date",

    title = "Climate oscilation indexes",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

## 1.6 PV

In [ ]:
#some code
df = pd.read_csv(os.path.join(data_folder, "df_pv.csv"))
df["size"] = 1

In [ ]:
df.head(10)

In [ ]:
fig = px.scatter_geo(
    df.loc[(df["date"] >= "1979-01-01") & (df["date"] <= "1979-02-01") & (df["level"] == 100)],
    lat="latitude",
    lon="longitude",
    color="speed",
    size = "size",
    color_continuous_scale = plt_style_s,
    size_max=7,
    opacity = 0.75,
    animation_frame = "date",
    labels = {"speed" : "wind speed [m/s]"},
    title = "Polar vortex"
    #zoom=10,
    )


#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 8.4, 'lat': 60},
        'style': "carto-positron",
        #'zoom': 3,
        #"projection": "albers usa",
    }
)

#fig.update_geos(projection_type="natural earth")

#update markers

scale_show(fig)

In [ ]:
fig = px.scatter_mapbox(
    df.loc[(df["date"] >= "1979-01-01") & (df["date"] <= "1979-02-01")],
    lat="latitude",
    lon="longitude",
    color="speed",
    size = "size",
    color_continuous_scale = plt_style_s,
    size_max=10,
    opacity = 0.5,
    animation_frame = "date",

    labels = {"speed" : "wind speed [m/s]"},
    )


#adjust view
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 8.4, 'lat': 60},
        'style': "carto-positron",
        'zoom': 3,
        #"projection": "albers usa",
    }
)

#fig.update_geos(projection_type="natural earth")

#update markers

scale_show(fig)

In [ ]:
plot_scaler = 15
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "1979-01-01") & (df["date"] <= "1979-01-07")],
    x = "longitude",
    y = "latitude",
    color = "speed",
    size = "size",
    size_max = 1 * plot_scaler - 3,
    opacity = 1,
    facet_col = "date",
    #animation_frame = "date",

    height = 45 * plot_scaler,
    width = (15) * plot_scaler * 6,
    color_continuous_scale  = plt_style_s,

    title = "Polar vortex wind speed",

    labels = {"speed" : "speed [m/s]"},
)


fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
plot_scaler = 15
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "1979-01-01") & (df["date"] <= "1979-01-07")],
    x = "longitude",
    y = "latitude",
    color = "t",
    size = "size",
    size_max = 1 * plot_scaler -3,
    opacity = 1,
    facet_col = "date",
    #animation_frame = "date",

    height = 45 * plot_scaler,
    width = (13 + 2) * plot_scaler * 6,
    color_continuous_scale  = plt_style_s,

    title = "Polar vortex temperatures",

    labels = {"t" : "t [░k]"},
)


fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
plot_scaler = 15
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "2019-03-01") & (df["date"] <= "2019-03-04")],
    y = "level",
    x = "latitude",
    color = "speed",
    size = "size",
    size_max = 1 * plot_scaler - 3,
    opacity = 1,
    facet_row = "date",
    #animation_frame = "date",

    height = (15 * 4)  * plot_scaler,
    width = 60 * plot_scaler,
    color_continuous_scale =  plt_style_s,

    title = "Polar vortex wind speed",

    labels = {"speed" : "speed [m/s]"},
)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
plot_scaler = 15
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "2019-03-01") & (df["date"] <= "2019-03-04")],
    y = "level",
    x = "latitude",
    color = "t",
    size = "size",
    size_max = 1 * plot_scaler - 3,
    opacity = 1,
    facet_row = "date",
    #animation_frame = "date",

    height = (15 * 4)  * plot_scaler,
    width = 60 * plot_scaler,
    color_continuous_scale =  plt_style_s,

    title = "Polar vortex temperature",

    labels = {"t" : "t [░k]"},
)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
del df

# 2. Data preparation

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#folders
data_folder = "data"

#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#set if data should be safed
save_data = False

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

## 2.1 Main data frame feature engineering

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df.csv"))
df.head()

In [ ]:
#clean up
df.drop(labels = [col for col in df.columns.tolist() if "unnamed" in col.lower()], axis = 1, inplace = True)

In [ ]:
day : int = 24 * 60 * 60 #[sec]
year : int = day * 366 #[sec] : 1,\n2020 was a leap year

year

In [ ]:
#creat col
df["timestamp"] = pd.to_datetime(df["date"]).apply(datetime.timestamp)

#calculate values
day : int = 24 * 60 * 60 #[sec]
year : int = day * 366 #[sec] : 1,\n2020 was a leap year

#set columns
df["year_sin"] = np.sin(df["timestamp"] * (2*np.pi / year))
df["year_cos"] = np.cos(df["timestamp"] * (2*np.pi / year))

#del unneedec col
df.drop(labels = ["timestamp"], axis = 1, inplace = True)
df.head()

In [ ]:
# add wind speed and direction
fig = px.line(
    data_frame = df.iloc[15000:],
    x = "date",
    y = ["year_sin", "year_cos"],

    title = "Year sine and cosine",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
# add rolling mean for ao, soi, t2m

cols_rolling_mean = ["soi", "ao", "mjo_amplitude", "t2m", "nao"]
offset = 30 #days

for col in cols_rolling_mean:

    #get rolling mean
    col_name = f"ma_{col}"
    df[col_name] = df[col].rolling(offset).mean()

    #create plot
    fig = px.line(
        data_frame = df[15000:],
        y = [col, col_name],
        x = "date",
        title = f"Moving average: {col}",
        color_discrete_sequence = plt_style_s
    )

    scale_show(fig)

In [ ]:
# add wind speed
df['wind_speed'] = np.sqrt(df['u10']**2 + df['v10']**2)

#add wind direction
df['wind_direction'] = np.rad2deg(np.arctan2(df['u10'], df['v10'])) % 360
df['wind_direction'] = (df['wind_direction'] + 90) % 360

df.head().T

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "wind_direction",
    histnorm = "probability density",
    title = "Wind direction",

    color = "month",
    barmode = "stack",
    opacity = 1,

    nbins = 180,

    labels = {"wind_direction" : "wind direction [░]"},

    color_discrete_sequence = plt_style_c,
)

scale_show(fig)

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "wind_speed",
    histnorm = "probability density",
    title = "wind_speed",

    color = "month",
    barmode = "stack",
    opacity = 1,

    nbins = 180,

    labels = {"wind_speed" : "wind speed [m/s]"},

    color_discrete_sequence = plt_style_c,
)

scale_show(fig)

In [ ]:
# add peak identifiers for mjo, ao, soi

In [ ]:
df.set_index(keys = "date", inplace = True)

if save_data:
    df.to_csv(os.path.join(data_folder, "df_fe.csv"))

## 2.2 Polar vortex index engineering

read some: https://www.severe-weather.eu/global-weather/strong-polar-vortex-warming-collapse-event-forecast-spring-2022-usa-europe-fa/

analyzing patterns during break down events

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_pv.csv"))
#df["date"] = pd.to_datetime(df["date"])
df["size"] = 1
df.head()

interpolate preassure levels

In [ ]:
#interplolate data for plotting
missing_levels = [40,60,80,90]

dates   = df["date"].unique()
lons    = df["longitude"].unique()
lats    = df["latitude"].unique()
lvls    = missing_levels

interpolation_data = {
    "date"          :[],
    "longitude"     :[],
    "latitude"      :[],
    "level"         :[],
}

#ugly and inefficient code

for date in dates:
    for lon in lons:
        for lat in lats:
            for lvl in lvls:

                interpolation_data["date"].append(date)
                interpolation_data["longitude"].append(lon)
                interpolation_data["latitude"].append(lat)
                interpolation_data["level"].append(lvl)

df_int = pd.DataFrame(interpolation_data)
df_int.head()

#merge dfs
df = pd.concat(objs = [df, df_int])
df.sort_values(by = ["date", "longitude", "latitude", "level"], inplace = True, axis = 0)

#clean up
df.reset_index(inplace = True)

#interpolate values

df["interpolated"] = 0
df.loc[df["level"].isin(missing_levels), "interpolated"] = 1

df.interpolate(metohd = "linear", inplace = True)

#recalculate speed and dircetion for interpolated values

#speed
df.loc[df["interpolated"] == 1, "speed"] = np.sqrt(df["u"] ** 2 + df["v"] ** 2)

df.loc[df["interpolated"] == 1, 'direction'] = np.rad2deg(np.arctan2(df['u'], df['v'])) % 360
df.loc[df["interpolated"] == 1, 'direction'] = (df['direction'] + 90) % 360

df.head(20)

In [ ]:
if save_data is True:
    df_pv_clustering = df[["date", "longitude", "latitude", "level", "t", "speed"]].loc[df["longitude"] == 8]
    df_pv_clustering.set_index("date", drop = True, inplace = True)
    df_pv_clustering.to_csv(os.path.join(data_folder, "df_pv_clustering.csv"))

In [ ]:
plot_scaler = 20
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "2019-03-01") & (df["date"] <= "2019-03-04")],
    y = "level",
    x = "latitude",
    color = "speed",
    size = "size",
    size_max = 1 * plot_scaler - 1,
    opacity = 1,
    facet_row = "date",
    #animation_frame = "date",

    height = (15 * 4)  * plot_scaler,
    width = 60 * plot_scaler,
    color_continuous_scale =  plt_style_s,

    title = "Polar vortex wind speed",

    labels = {"speed" : "speed [m/s]"},
)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
plot_scaler = 20
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "2019-03-01") & (df["date"] <= "2019-03-04")],
    y = "level",
    x = "latitude",
    color = "t",
    size = "size",
    size_max = 1 * plot_scaler - 1,
    opacity = 1,
    facet_row = "date",
    #animation_frame = "date",

    height = (15 * 4)  * plot_scaler,
    width = 60 * plot_scaler,
    color_continuous_scale =  plt_style_s,

    title = "Polar vortex temperature",

    labels = {"t" : "t [k░]"},
)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

In [ ]:
#drop latitude 90. Looks like a cut in the data
df.drop(df.loc[df["latitude"] == 90].index, inplace = True)


plot_scaler = 20
fig = px.scatter(
    data_frame = df.loc[(df["date"] >= "1979-01-23") & (df["date"] <= "1979-01-30")],
    y = "level",
    x = "latitude",
    color = "speed",
    size = "size",
    size_max = 1 * plot_scaler - 1,
    opacity = 1,
    facet_row = "date",
    #animation_frame = "date",

    height = (20 * 6)  * plot_scaler,
    width = 60 * plot_scaler,
    color_continuous_scale =  plt_style_s,

    title = "Polar vortex wind speed",

    labels = {"speed" : "speed [m/s]"},
)

fig['layout']['yaxis']['autorange'] = "reversed"

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)

fig.show()

Costuom index 1 (wind speed only)
 - Convolution on lattitide level (vertical bar with pieces)
 - Detect border of vortex by wind speeds (get most constant latitude by std, invert it, multiply by wind speed)
 - value_0 = border_height (0 .. n, 0 = switzerland, n = north pole)
 - value_1 = temperature  at height

In [ ]:
def pv_i1_w(df):

    #drop unneded cols
    df_i1 = df[["date", "latitude", "t", "speed"]]

    #crate latitude aggregation
    df_i1 = df_i1.groupby(by = ["date", "latitude"], as_index = False).aggregate(
        speed_mean = ("speed", "mean"),
        speed_std = ("speed", "std"),
        t_mean =    ("t", "mean"),
    )

    #normalize the standard deviation
    # source: https://business.blogthinkbig.com/warning-about-normalizing-data/

    #get local min and max
    df_i1_minmax = df_i1[["date", "speed_std"]]
    df_i1_minmax = df_i1_minmax.groupby(by = ["date"], as_index = False).aggregate(
        speed_std_max = ("speed_std", "max"),
        speed_std_min = ("speed_std", "min"),
    )

    #append data together
    df_i1["date"] = df_i1["date"].astype(str)
    df_i1_minmax["date"] = df_i1_minmax["date"].astype(str)

    df_i1["date"] = pd.to_datetime(df_i1["date"])
    df_i1_minmax["date"] = pd.to_datetime(df_i1_minmax["date"])

    df_i1 = df_i1.merge(right = df_i1_minmax, on = "date", how = "left")

    #calculate normalized std
    df_i1["speed_norm_inv"] = 1 - (df_i1["speed_std"] - df_i1["speed_std_min"]) / (df_i1["speed_std_max"] - df_i1["speed_std_min"])

    #calculate weighted wind speed
    df_i1["speed_weight"] = df_i1["speed_mean"] * df_i1["speed_norm_inv"]

    #get local max and min for index based on date
    df_i1_max = df_i1[["date", "speed_weight"]].groupby(by = "date", as_index = False).max()
    df_i1_max["is_max"] = 1

    #get the lattitude at max
    df_i1 = df_i1.merge(right = df_i1_max, on = ["date", "speed_weight"], how = "left")
    df_i1 = df_i1.loc[df_i1["is_max"] == 1]

    #clean up
    df_i1 = df_i1[["date", "latitude", "t_mean"]]
    df_i1_w = df_i1; del df_i1

    return df_i1_w

In [ ]:
df_i1_w = pv_i1_w(df)
df_i1_w.head()

In [ ]:
def plot_i1(df, df_i1, plot_param):

    dates = [
        "2019-03-01", "2019-03-02", "2019-03-03", "2019-03-04"
        #"1979-04-01","1979-05-01","1979-06-01",
        #"1979-07-01","1979-08-01","1979-09-01",
        #"1979-10-01","1979-11-01","1979-12-01",
    ]

    if plot_param == "t":
        title = "temperature"
        widht_correction = 2
    elif plot_param == "speed":
        title = "wind speed"
        widht_correction = 4

    for date in dates:

        plot_scaler = 20

        fig = px.scatter(
            data_frame = df.loc[df["date"] == date],
            x = "latitude",
            y = "level",
            color = plot_param,
            size = "size",
            size_max = 1 * plot_scaler -7,
            opacity = 1,
            facet_col = "date",
            #animation_frame = "date",

            height = 20  * plot_scaler,
            width = 60 * plot_scaler,
            color_continuous_scale =  plt_style_s,


            title = f"Polar vortex {title}",

            labels = {"speed" : "speed [m/s]"},
        )

        fig.update_traces(
            marker=dict(symbol="square",),
            selector=dict(mode='markers')
        )

        #indicator
        x = float(df_i1.loc[df_i1["date"] == date]["latitude"])

        fig.add_vline(
            x = x
        )

        #reverse axis
        fig['layout']['yaxis']['autorange'] = "reversed"

        fig.show()

In [ ]:
plot_i1(df, df_i1_w, "t")

Costuom index 1 (wind speed + tempereautre)
 - Convolution on lattitide level (vertical bar with pieces)
 - Detect border of vortex by wind speeds and temperature (get most constant latitude by std, invert it, multiply by wind speed)
 - value_0 = border_height (0 .. n, 0 = switzerland, n = north pole)
 - value_1 = temperature  at height

In [ ]:
def pv_i1_wt(df, speed_weight = 0.3, t_weight = 0.7):

    #drop unneded cols
    df_i1 = df[["date", "latitude", "t", "speed"]]

    #crate latitude aggregation
    df_i1 = df_i1.groupby(by = ["date", "latitude"], as_index = False).aggregate(
        speed_mean = ("speed", "mean"),
        speed_std = ("speed", "std"),
        t_mean =    ("t", "mean"),
    )

    #normalize the standard deviation
    # source: https://business.blogthinkbig.com/warning-about-normalizing-data/

    #get local min and max
    df_i1_minmax = df_i1[["date", "speed_std", "t_mean"]]
    df_i1_minmax = df_i1_minmax.groupby(by = ["date"], as_index = False).aggregate(
        speed_std_max   = ("speed_std", "max"),
        speed_std_min   = ("speed_std", "min"),
        t_mean_max      = ("t_mean", "max"),
        t_mean_min      = ("t_mean", "min"),
    )

    #append data together
    df_i1["date"] = df_i1["date"].astype(str)
    df_i1_minmax["date"] = df_i1_minmax["date"].astype(str)

    df_i1["date"] = pd.to_datetime(df_i1["date"])
    df_i1_minmax["date"] = pd.to_datetime(df_i1_minmax["date"])

    df_i1 = df_i1.merge(right = df_i1_minmax, on = "date", how = "left")

    #calculate normalized std
    df_i1["speed_norm_inv"] = 1 - (df_i1["speed_std"] - df_i1["speed_std_min"]) / (df_i1["speed_std_max"] - df_i1["speed_std_min"])
    df_i1["t_norm_inv"] = 1 - (df_i1["t_mean"] - df_i1["t_mean_min"]) / (df_i1["t_mean_max"] - df_i1["t_mean_min"])

    #calculate weighted wind speed
    df_i1["weight"] = df_i1["speed_mean"] * (df_i1["speed_norm_inv"] * speed_weight + df_i1["t_norm_inv"] * t_weight)

    #get local max and min for index based on date
    df_i1_max = df_i1[["date", "weight"]].groupby(by = "date", as_index = False).max()
    df_i1_max["is_max"] = 1

    #get the lattitude at max
    df_i1 = df_i1.merge(right = df_i1_max, on = ["date", "weight"], how = "left")
    df_i1 = df_i1.loc[df_i1["is_max"] == 1]

    #clean up
    df_i1 = df_i1[["date", "latitude", "t_mean"]]
    df_i1_wt = df_i1; del df_i1

    return df_i1_wt

In [ ]:
df_i1_wt = pv_i1_wt(df)
df_i1_wt.head()

In [ ]:
plot_i1(df, df_i1_wt, "speed") #"speed", "t"

In [ ]:
del df_i1_w, df_i1_wt

 Custom index 2 (edge detection):
 - detect sharp borders of temperature changes
 - create a contrast values accors n vertical rows
 - detect sharpest border, based on threshold
 - if no value recheas the threshold, a breakdown can be detected

In [ ]:
# Custom index idea 2:
# - detect sharp borders of temperature changes
# - create a contrast values accors n vertical rows
# - detect sharpest border, based on threshold
# - if no value recheas the threshold, a breakdown can be detected

In [ ]:
class PV_ind_2_v2():

    def main(df, n_lat, threshold, break_down_offset, break_down_sensitivity, metric):
        """metric: ["t", "speed"]"""

        df_i2 = PV_ind_2_v2.get_delta(df, n_lat, metric)
        df_i2 = PV_ind_2_v2.get_local_max_delta(df_i2)
        df_i2 = PV_ind_2_v2.apply_threshold(df_i2, threshold)
        df_i2 = PV_ind_2_v2.detect_breakdown(df_i2, break_down_offset, break_down_sensitivity)

        return df_i2

    def get_delta(df, n, metric):

        #drop unneded cols
        df_i2 = df[["date", "latitude", "t", "speed"]]
        del df

        #crate latitude aggregation
        df_i2 = df_i2.groupby(by = ["date", "latitude"], as_index = False).aggregate(
            speed_mean  = ("speed", "mean"),
            #speed_std   = ("speed", "std"),
            t_mean      = ("t", "mean"),
            #t_std       = ("t", "std"),
        )

        df_i2.sort_values(by = ["date", "latitude"], ascending = [True , False], inplace = True)

        #create offshift for border detection
        cols = [f"{metric}_mean"]

        for i in range(1, n+1):
            df_i2[f"{metric}_mean_-{i}"] = df_i2[f"{metric}_mean"].shift(-i)
            cols.append(f"{metric}_mean_{-i}")

        #create deltas for border detection (square values to only get positive values and highlight bigger deltas)
        cols_delta = []
        for i in range(n):
            df_i2[f"delta_{i}"] = abs(df_i2[cols[i]] - df_i2[cols[i + 1]])
            cols_delta.append(f"delta_{i}")

        #sum deltas to get border value
        df_i2["delta"] = df_i2[cols_delta].mean(axis=1)

        #drop lower n cols
        lats = df_i2["latitude"].unique()
        lats.sort()
        lats = lats[:n]

        df_i2.loc[df_i2["latitude"].isin(lats), "delta"] = None

        #clean up
        df_i2.drop(labels = cols[1:] + cols_delta, axis = 1, inplace = True)

        return df_i2

    def get_local_max_delta(df_i2):

        #get max values
        df_i2_max = df_i2[["date", "delta"]].groupby(by = ["date"], as_index = False).max()
        df_i2_max["is_max"] = 1

        #set max in master df
        df_i2 = df_i2.merge(right = df_i2_max, on = ["date", "delta"], how = "left")
        #df_i2["is_max"] = df_i2["is_max"].fillna(0)
        #df_i2["is_max"] = df_i2["is_max"].astype(int)

        #clean up
        del df_i2_max
        return df_i2

    def apply_threshold(df_i2, threshold):

        #set th as multiplicator
        th = threshold + 1

        #get mean
        df_i2_d = df_i2[["date","delta"]].groupby(by = "date", as_index = False).aggregate(
            mean_delta = ("delta", "mean"),
        )

        #combine and apply th
        df_i2 = df_i2.merge(right = df_i2_d, on = "date", how = "left")
        df_i2["mean_delta"] = df_i2["mean_delta"] * th

        #compare
        df_i2["pv_edge"] = 0
        df_i2.loc[(df_i2["is_max"] == 1) & (df_i2["delta"] > df_i2["mean_delta"]), "pv_edge"] = 1

        #clean up
        df_i2.dropna(subset = "is_max", inplace = True)
        df_i2.drop(labels = ["is_max"], axis = 1, inplace = True)

        return df_i2

    def detect_breakdown(df_i2, break_down_offset, break_down_sensitivity):

        #considered months
        considered_months = [12,1,2,3,4]
        df_i2["date"] = pd.to_datetime(df_i2["date"])

        #consider offset by applying rolling mean.
        df_i2["pv_edge_offset"] = df_i2["pv_edge"].rolling(break_down_offset).mean()

        #break down event

        df_i2["pv_break_down_event"] = 0
        df_i2.loc[
            (df_i2["pv_edge_offset"] <= break_down_sensitivity) &
            (df_i2["pv_edge_offset"].shift(1) > break_down_sensitivity) &
            (df_i2["date"].dt.month.isin(considered_months))
            , "pv_break_down_event"] = 1

        #clean up
        df_i2.drop(labels = "pv_edge_offset", axis = 1, inplace = True)

        return df_i2

In [ ]:
#params
n_lat                   = 3 #lat offset for calultion
threshold               = 1 #in % for vortex, multiple which has to be exceeded from popluation mean, to be counted as a pv edge
break_down_offset       = 3 #last n span, used to detect a break down
break_down_sensitivity  = 0.1 # in %. Sets the barrier or threshold, at which the value crossing it, a breakdown will be detected

df_i2_s = PV_ind_2_v2.main(
    df = df,
    n_lat = n_lat,
    threshold = threshold,
    break_down_offset = break_down_offset,
    break_down_sensitivity = break_down_sensitivity,
    metric = "speed",
)

df_i2_s.head(5)

In [ ]:
#params
n_lat                   = 2 #lat offset for calultion
threshold               = 1 #in % for vortex, multiple which has to be exceeded from popluation mean, to be counted as a pv edge
break_down_offset       = 3 #last n span, used to detect a break down
break_down_sensitivity  = 0.1 # in %. Sets the barrier or threshold, at which the value crossing it, a breakdown will be detected

df_i2_t = PV_ind_2_v2.main(
    df = df,
    n_lat = n_lat,
    threshold = threshold,
    break_down_offset = break_down_offset,
    break_down_sensitivity = break_down_sensitivity,
    metric = "t",
)

df_i2_t.head(5)

In [ ]:

def plot_i2(df, df_i, plot_param, dates = None):

    if dates is None:

        dates = [
            "2019-03-01", "2019-03-02", "2019-03-03", "2019-03-04"
            #"2022-04-01","2022-05-01","2022-06-01",
            #"2022-07-01","2022-08-01","2022-09-01",
            #"2022-10-01","2022-11-01","2022-12-01",
        ]

    if plot_param == "t":
        title = "temperature"
    elif plot_param == "speed":
        title = "wind speed"

    for date in dates:

        plot_scaler = 20

        fig = px.scatter(
            data_frame = df.loc[df["date"] == date],
            x = "latitude",
            y = "level",
            color = plot_param,
            size = "size",
            size_max = 1 * plot_scaler - 3,
            opacity = 1,
            facet_col = "date",
            #animation_frame = "date",

            height = 20  * plot_scaler,
            width = 60 * plot_scaler,
            color_continuous_scale  = plt_style_s,

            title = f"Polar vortex: {title}",

            labels = {"speed" : "speed [m/s]", "t" : "t [░k]"},
        )

        fig.update_traces(
            marker=dict(symbol="square",),
            selector=dict(mode='markers')
        )

        #indicator
        x = float(df_i.loc[df_i["date"] == date]["latitude"])
        #th_exceeded = int(df_i1.loc[df_i1["date"] == date]["pv_edge"])

        fig['layout']['yaxis']['autorange'] = "reversed"

        fig.add_vline(
            x = x,
            #line_color = "black",
            #line_width = th_exceeded * 5,
        )

        fig.show()

In [ ]:
df

In [ ]:
plot_i2(df, df_i2_s, "speed")

In [ ]:
plot_i2(df, df_i2_t, "t")

In [ ]:
fig = px.line(
    data_frame = df_i2_s.iloc[-5000:],
    x = "date",
    y = "pv_break_down_event",

    title = "Polar vortex edge detection",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

Custom index 3 (delta index):
 - convolution of data at each end with weighted mean, based on distance (twice, one for wind speed, and one for temperature)
 - get delta of the area as a single value for each metric from the two obainted values
 - or leave the two values as they are

In [ ]:
class PV_ind_3():

    def main(df, metric):

        df_i3 = PV_ind_3.agg(df, metric)
        df_i3 = PV_ind_3.set_weights(df_i3)
        df_i3 = PV_ind_3.normalize_metric(df_i3, metric)
        df_i3 = PV_ind_3.set_p_values(df_i3, metric)

        return df_i3

    def agg(df, metric):

        #aggreagte data
        df_i3 = df[["date", "latitude", metric]].groupby(by = ["date", "latitude"], as_index = False).mean()
        #df_i3.rename(mapper = {metric : "metric"}, inplace = True, axis = 1)

        return df_i3

    def normalize_metric(df_i3, metric):

        #get local min and max
        df_i3_minmax = df_i3[["date", metric]]
        df_i3_minmax = df_i3_minmax.groupby(by = "date").aggregate(
            metric_max      = (metric, "max"),
            metric_min      = (metric, "min"),
        )

        #append data together
        df_i3 = df_i3.merge(right = df_i3_minmax, on = "date", how = "left")

        #calculated normalized metric
        df_i3[f"{metric}_norm"] = (df_i3[metric] - df_i3["metric_min"]) / (df_i3["metric_max"] - df_i3["metric_min"])

        #clean up
        df_i3.drop(labels = ["metric_max", "metric_min"], axis = 1, inplace = True)

        return df_i3

    def set_weights(df_i3):

        #transform
        min_lat = float(df_i3["latitude"].min())
        df_i3["weight_n"] = df_i3["latitude"] - min_lat

        #transform to values between 0 and 1
        max_lat = float(df_i3["latitude"].max())

        df_i3["weight_n"] = df_i3["weight_n"] / max_lat
        df_i3["weight_s"] = 1 - df_i3["weight_n"]

        return df_i3

    def set_p_values(df_i3, metric):

        #get p_value
        df_i3["p_north"] = df_i3[f"{metric}_norm"] * df_i3["weight_n"]
        df_i3["p_south"] = df_i3[f"{metric}_norm"] * df_i3["weight_s"]

        #get sum
        df_i3.drop(labels = [metric, "weight_n", "weight_s", f"{metric}_norm", "latitude"], axis = 1, inplace = True)
        df_i3 = df_i3.groupby(by = ["date"], as_index = False).sum()

        #get delta
        df_i3["p_delta"] = df_i3["p_south"] - df_i3["p_north"]

        return df_i3

In [ ]:
df_i3 = PV_ind_3.main(df, metric = "t")
df_i3.head(10)

In [ ]:
def plot_i3(df, df_i3, plot_param):

    dates = [
        "2019-03-01", "2019-03-02", "2019-03-03", "2019-03-04",
        #"2022-04-01","2022-05-01","2022-06-01",
        #"2022-07-01","2022-08-01","2022-09-01",
        #"2022-10-01","2022-11-01","2022-12-01",
    ]

    if plot_param == "t":
        title = "temperature"
        widht_correction = 2
    elif plot_param == "speed":
        title = "wind speed"
        widht_correction = 4


    for date in dates:

        plot_scaler = 19

        #indicator
        p_delta = round(float(df_i3.loc[df_i3["date"] == date]["p_delta"]),2)

        fig = px.scatter(
            data_frame = df.loc[df["date"] == date],
            x = "latitude",
            y = "level",
            color = plot_param,
            size = "size",
            size_max = 1 * plot_scaler - 3,
            opacity = 1,
            facet_col = "date",
            #animation_frame = "date",

            height = 20 * plot_scaler,
            width = 60 * plot_scaler,
            color_continuous_scale  = plt_style_s,

            title = f"Polar vortex: {title}\nP_delta: {p_delta}",

            labels = {"speed" : "speed [m/s]", "t" : "t [░k]"},
        )

        fig.update_traces(
            marker=dict(symbol="square",),
            selector=dict(mode='markers')
        )

        fig['layout']['yaxis']['autorange'] = "reversed"

        fig.show()

In [ ]:
plot_i3(df = df, df_i3 = df_i3, plot_param = "t")

In [ ]:
df_i3.head()

In [ ]:
fig = px.line(
    data_frame = df_i3.iloc[-5000:],
    x = "date",
    y = "p_delta",

    color_discrete_sequence = plt_style_s,
    title = "Index 3: Temperature",
)

scale_show(fig)

Custom index 4 (local max):
 - get the local max, though all pressure levels

In [ ]:
def pv_ind_4(df, metric, agg):
    """
    metric = ["speed", "t"]
    agg = ["sum", "max", "mean", "median"]
    """

    #drop unneded cols
    df_i4 = df[["date", "latitude", "t", "speed"]]

    #crate latitude aggregation
    df_i4 = df_i4.groupby(by = ["date", "latitude"], as_index = False).aggregate(
        speed_agg = ("speed", agg),
        t_agg =    ("t", agg),
    )

    #normalize the standard deviation
    # source: https://business.blogthinkbig.com/warning-about-normalizing-data/

    #get local max for metric and agg
    func = {"speed" : "max", "t" : "min"}

    df_i4_max = df_i4[["date",f"{metric}_agg"]]
    df_i4_max = df_i4_max.groupby(by = ["date"], as_index = False).aggregate(
        metric_max = (f"{metric}_agg", func[metric]),
    )

    #append data together
    df_i4["date"] = df_i4["date"].astype(str)
    df_i4_max["date"] = df_i4_max["date"].astype(str)

    df_i4["date"] = pd.to_datetime(df_i4["date"])
    df_i4_max["date"] = pd.to_datetime(df_i4_max["date"])

    df_i4 = df_i4.merge(right = df_i4_max, on = ["date"], how = "left")

    #only get matchin max value
    df_i4 = df_i4.loc[df_i4["metric_max"] == df_i4[f"{metric}_agg"]]
    df_i4.drop(labels = "metric_max", axis = 1, inplace = True)

    return df_i4


In [ ]:
df_i4_s = pv_ind_4(df, "speed", "mean")
df_i4_s.head()

In [ ]:
plot_i2(df, df_i4_s, "speed")

In [ ]:
df_i4_t = pv_ind_4(df, "t", "mean")
df_i4_t.head()

In [ ]:
plot_i2(df, df_i4_t, "t")

In [ ]:
df_i4_t["type"] = "temp"
df_i4_s["type"] = "speed"

df_i4 = pd.concat(objs = [df_i4_t, df_i4_s])
df_i4.sort_values(by = "date", ascending = True, inplace = True)

df_i4.head()

In [ ]:
fig = px.line(
    data_frame  = df_i4.iloc[-2000:],
    x = "date",
    y = "latitude",
    title = "Polar vortex: Index 4",
    color_discrete_sequence = plt_style_s,
    color = "type",
)


scale_show(fig)

In [ ]:
#temperature: index 2
#wind speed: index 4

Custom index 5 (SSW)

## 2.3 Index optimization and setting
Sudden stratospheric warming events: https://csl.noaa.gov/groups/csl8/sswcompendium/majorevents.html

### 2.3.1 PVT: Polar vortex temp (index 2)

In [ ]:
def plot_i_breakdown(df, plot_param, break_down_date):

    if plot_param == "t":
        title = "temperature"
    elif plot_param == "speed":
        title = "wind speed"

    plot_scaler = 19

    fig = px.scatter(
        data_frame = df,
        x = "latitude",
        y = "level",
        color = plot_param,
        size = "size",
        size_max = 1 * plot_scaler - 1,
        opacity = 1,
        facet_row = "date",

        height = 15 * plot_scaler * len(df["date"].unique().tolist()),
        width = 60 * plot_scaler,
        color_continuous_scale  = plt_style_s,

        title = f"Polar vortex breakdown - {break_down_date}: {title}",

        labels = {"speed" : "speed [m/s]", "t" : "t [k]"},
    )

    fig.update_traces(
        marker=dict(symbol="square",),
        selector=dict(mode='markers')
    )

    fig['layout']['yaxis']['autorange'] = "reversed"

    fig.show()

In [ ]:
del df_i2_t

In [ ]:
#optimize temperatrue (i2)
#params
n_lat                   = 3 #lat offset for calultion
threshold               = 1.0 #in % for vortex, multiple which has to be exceeded from popluation mean, to be counted as a pv edge
break_down_offset       = 15 #last n span, used to detect a break down
break_down_sensitivity  = 0.1 # in %. Sets the barrier or threshold, at which the value crossing it, a breakdown will be detected

df_i2_t = PV_ind_2_v2.main(
    df = df,
    n_lat = n_lat,
    threshold = threshold,
    break_down_offset = break_down_offset,
    break_down_sensitivity = break_down_sensitivity,
    metric = "t",
)

df_i2_t.head(5)

In [ ]:
n = -1000

fig = px.line(
    data_frame = df_i2_t,
    x = "date",
    y = "pv_break_down_event",
    title = "PV break down events i2: Temperature",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)


In [ ]:
#add momth
df_i2_t["month"] = pd.DatetimeIndex(df_i2_t["date"]).month


#plot porbaibilty per month
fig = px.histogram(
    data_frame = df_i2_t,
    x = "month",
    y = "pv_break_down_event",
    histfunc = "sum",
    barmode = "stack",
    title = "PV break down events i2 per month: Temperature",
    color_discrete_sequence = plt_style_s,

    width = 700,
    height = 700,
)

fig.show()

In [ ]:
#add momth
df_i2_t["month"] = pd.DatetimeIndex(df_i2_t["date"]).month


#plot porbaibilty per month
fig = px.histogram(
    data_frame = df_i2_t,
    x = "month",
    y = "pv_edge",
    histfunc = "sum",
    barmode = "stack",
    title = "PV edges i2: Temperature",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
#plot breakdonw events
df_i2_t.reset_index(inplace = True, drop = True)

plot_range = 5

df_i2_t["date"] = df_i2_t["date"].astype(str) #whyyy?

for i in df_i2_t.loc[df_i2_t["pv_break_down_event"] == 1].index[-4:]:
    for param in ["t", "speed"]:

        print(i)
        break_down_date = df_i2_t.iloc[i]["date"]

        offset = int(plot_range / 2)
        start_date      = df_i2_t.iloc[i - offset]["date"]
        end_date        = df_i2_t.iloc[i + offset]["date"]


        df_plot = df.loc[(df["date"] >= start_date) & (df["date"] <= end_date)]
        plot_i_breakdown(df = df_plot, plot_param = param, break_down_date = break_down_date)

In [ ]:
#prepare data frame for merging
unneded_cols = ["speed_mean", "delta", "mean_delta", "month"]

for col in unneded_cols:

    try:
        df_i2_t.drop(labels = col, axis = 1, inplace = True)
    except:
        print(f"{col} does not exist")

#add prefix to generate unique values
df_i2_t = df_i2_t.add_prefix("pvt_") #polar vortex temperature

df_i2_t.head()

### 2.3.2 PVS: Polar vortex wind speed (index 4)

In [ ]:
class PV_ind_4():

    def main(df, metric, threshold, break_down_offset, break_down_sensitivity):

        df_i4 = PV_ind_4.get_local_max(df, metric)
        df_i4 = PV_ind_4.apply_threshold(df_i4, df, threshold, metric)
        df_i4 = PV_ind_4.detect_breakdown(df_i4, break_down_offset, break_down_sensitivity)

        return df_i4

    def get_local_max(df, metric):

        #drop unneded cols
        df_i4 = df[["date", "latitude", "t", "speed"]]

        #crate latitude aggregation
        df_i4 = df_i4.groupby(by = ["date", "latitude"], as_index = False).aggregate(
            speed_mean = ("speed", "mean"),
            t_mean =    ("t", "mean"),
        )

        #get local max for metric and agg
        func = {"speed" : "max", "t" : "min"}

        df_i4_max = df_i4[["date",f"{metric}_mean"]]
        df_i4_max = df_i4_max.groupby(by = ["date"], as_index = False).aggregate(
            metric_max = (f"{metric}_mean", func[metric]),
        )

        #append data together
        df_i4["date"] = df_i4["date"].astype(str)
        df_i4_max["date"] = df_i4_max["date"].astype(str)

        df_i4["date"] = pd.to_datetime(df_i4["date"])
        df_i4_max["date"] = pd.to_datetime(df_i4_max["date"])

        df_i4 = df_i4.merge(right = df_i4_max, on = ["date"], how = "left")

        #only get matching max value
        df_i4 = df_i4.loc[df_i4["metric_max"] == df_i4[f"{metric}_mean"]]
        df_i4.drop(labels = "metric_max", axis = 1, inplace = True)

        #drop unneded metric
        unneded_metric = ["t", "speed"]
        unneded_metric.remove(metric)
        df_i4.drop(labels = f"{unneded_metric[0]}_mean", axis = 1, inplace = True)

        return df_i4

    def apply_threshold(df_i4, df, threshold, metric):

        #only get needed cols
        df_i4_mean = df[["date", metric]]

        #set th as multiplicator
        th = threshold + 1

        #get overall mean metric
        df_i4_mean = df_i4_mean.groupby(by = "date", as_index = False).mean()

        #stupid date time formats keep changing
        df_i4_mean['date'] = df_i4_mean['date'].astype('datetime64[ns]')

        #merge and apply th
        df_i4 = df_i4.merge(right = df_i4_mean, on = "date", how = "left")
        df_i4[metric] = df_i4[metric] * th

        #check if th is exceeded
        df_i4["pv_edge"] = 0
        df_i4.loc[df_i4[f"{metric}_mean"] > df_i4[metric], "pv_edge"] = 1

        #drop unnded cols
        df_i4.drop(labels = [metric], inplace = True, axis = 1)

        return df_i4

    def detect_breakdown(df_i4, break_down_offset, break_down_sensitivity):

        #relevant months
        considered_months = [12,1,2,3,4]

        #consider offset by applying rolling mean.
        df_i4["pv_edge_offset"] = df_i4["pv_edge"].rolling(break_down_offset).mean()

        #break down event
        df_i4["pv_break_down_event"] = 0
        df_i4.loc[
            (df_i4["pv_edge_offset"] <= break_down_sensitivity) &
            (df_i4["pv_edge_offset"].shift(1) > break_down_sensitivity) &
            (df_i4["date"].dt.month.isin(considered_months))
            , "pv_break_down_event"] = 1

        #clean up
        df_i4.drop(labels = "pv_edge_offset", axis = 1, inplace = True)

        return df_i4


In [ ]:
threshold               = 0.5
break_down_offset       = 60 #last n span, used to detect a break down
break_down_sensitivity  = 0.875 # in %. Sets the barrier or threshold, at which the value crossing it, a breakdown will be detected

df_i4_s = PV_ind_4.main(
    df = df,
    metric = "speed",

    threshold = threshold,
    break_down_offset = break_down_offset,
    break_down_sensitivity = break_down_sensitivity,
)

In [ ]:
df_i4_s.head()

In [ ]:
n = -1000

fig = px.line(
    data_frame = df_i4_s,
    x = "date",
    y = "pv_break_down_event",
    title = "PV break down events i4: Wind speed",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
#add momth
df_i4_s["month"] = pd.DatetimeIndex(df_i4_s["date"]).month


#plot porbaibilty per month
fig = px.histogram(
    data_frame = df_i4_s,
    x = "month",
    y = "pv_break_down_event",
    histfunc = "sum",
    barmode = "stack",
    title = "PV break down events i4 per month: Wind speed",
    color_discrete_sequence = plt_style_s,

    height = 700,
    width = 700,
)

fig.show()

In [ ]:
#add momth
df_i4_s["month"] = pd.DatetimeIndex(df_i4_s["date"]).month


#plot porbaibilty per month
fig = px.histogram(
    data_frame = df_i4_s,
    x = "month",
    y = "pv_edge",
    histfunc = "sum",
    barmode = "stack",
    title = "PV edges i4: Temperature",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
df_i4_s.info()

In [ ]:
#plot breakdonw events
df_i4_s.reset_index(inplace = True, drop = True)
df_i4_s["date"] = df_i4_s["date"].astype(str)

plot_range = 5

for i in df_i4_s.loc[df_i4_s["pv_break_down_event"] == 1].index[-4:]:
    for param in ["t", "speed"]:

        print(i)
        break_down_date = "2022-03-05" #df_i4_s.iloc[i]["date"]

        offset = int(plot_range / 2)
        start_date      = "2022-03-03"#df_i4_s.iloc[i - offset]["date"]
        end_date        = "2022-03-07"#df_i4_s.iloc[i + offset]["date"]

        df_plot = df.loc[(df["date"] >= start_date) & (df["date"] <= end_date)]
        plot_i_breakdown(df = df_plot, plot_param = param, break_down_date = break_down_date)

In [ ]:
#remove unneded cols
try:
    df_i4_s.drop(labels = "month", axis = 1, inplace = True)
except:
    print("month col does not exist")

#add prefix
df_i4_s = df_i4_s.add_prefix("pvs_")

#check
df_i4_s.head()

### 2.3.3 Merge and compare

In [ ]:
#merge data
df_pv = df_i2_t.merge(right = df_i4_s, left_on = "pvt_date", right_on = "pvs_date", how = "left")

#clean up
df_pv.drop(labels = "pvs_date", axis = 1, inplace = True)
df_pv.rename(mapper = {"pvt_date" : "date"}, axis = 1, inplace = True)

df_pv.head()

In [ ]:
#some plots and correlation matrix
df_pv_plot = df_pv.copy()

df_pv_plot["size"] = 1


fig = px.scatter(
    data_frame = df_pv_plot,
    x = "pvs_latitude",
    y = "pvt_latitude",
    title = "Temperature and wind speed comparison: latitude",
    color_discrete_sequence = plt_style_s,
    opacity = 0.05,
    size_max = 17,
    size = "size",

    height = 1000,
    width = 1000,

)

fig.update_traces(
    marker=dict(symbol="square",),
    selector=dict(mode='markers')
)


fig.show()

In [ ]:
fig = px.line(
    data_frame = df_pv_plot,
    x = "date",
    y = ["pvs_pv_break_down_event", "pvt_pv_break_down_event"],

    title = "Break down events",
    color_discrete_sequence = plt_style_s,

    labels = {"value" : ""}
)

newnames = {"pvs_pv_break_down_event" : "pvs breakdwon", "pvt_pv_break_down_event" : "pvt breakdown"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df_pv_plot.iloc[-2000:],
    x = "date",
    y = ["pvs_pv_edge", "pvt_pv_edge"],

    title = "PV edges",
    color_discrete_sequence = plt_style_s,

    labels = {"value" : ""}
)

newnames = {"pvs_pv_edge" : "pvs edge", "pvt_pv_edge" : "pvt edge"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

scale_show(fig)

In [ ]:
df_pv

In [ ]:
#merge with main dataframe
df_fe = pd.read_csv(os.path.join(data_folder, "df_fe.csv"))
df_fe.head(2)

In [ ]:
print(f"df_fe: {df_fe.shape}\ndf_pv: {df_pv.shape}")

In [ ]:
df_fe = df_fe.merge(right = df_pv, left_on = "date", right_on = "date")


In [ ]:
df_fe.head().T

In [ ]:
#save main data frame with added pv data
if save_data is True:
    df_fe.set_index("date", drop = True, inplace = True)
    df_fe.to_csv(os.path.join(data_folder, "df_fe.csv"))

## 2.4 Categorizing data for ml models

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_fe.csv"))
df["date"] = pd.DataFrame(df["date"])
df.head().T

In [ ]:
for month in df["month"].unique()[:3]:

    fig = px.box(
        data_frame = df.loc[(df["month"] == month)],
        x = "day",
        y = "t2m",

        title = f"Temperature by month: {month}",
        color_discrete_sequence = plt_style_s,

        width = size["width"],
        height = size["height"] * 12,
    )

    scale_show(fig)


In [ ]:
#calculate fortnight periods 1 and 2
df["t2m_t1"] = df["t2m"].rolling(14).mean().shift(-14)
df["t2m_t2"] = df["t2m"].rolling(14).mean().shift(-28)

#set category
df_median = df[["month", "day", "t2m_t1", "t2m_t2"]].groupby(by = ["month", "day"], as_index = False).aggregate(
    t2m_t1_mean = ("t2m_t1", "mean"),
    t2m_t2_mean = ("t2m_t2", "mean"),
)

#merge df to main
df = df.merge(right = df_median, on = ["month", "day"])
df.sort_values(by = "date", inplace = True, ascending = True)

#set, if value is above or below median
df["t2m_t1_cat"] = 0
df["t2m_t2_cat"] = 0

df.loc[df["t2m_t1"] >= df["t2m_t1_mean"], "t2m_t1_cat"] = 1
df.loc[df["t2m_t2"] >= df["t2m_t2_mean"], "t2m_t2_cat"] = 1

#see df
df.head(15).T

In [ ]:
fig = px.histogram(
    data_frame = df,
    x = "year",
    y  = "t2m_t1_cat",
    color = "t2m_t1_cat",
    color_discrete_sequence = plt_style_s,
    title = "Target variable distribution",
    histfunc = "count",
    barmode= "stack"
)

scale_show(fig)

In [ ]:
df_target = df[["t2m_t1_cat", "t2m_t2_cat"]]
df_target["same_cat"] = df_target["t2m_t1_cat"] == df_target["t2m_t2_cat"]

fig = px.histogram(
    data_frame = df_target,
    x = "same_cat",
    color = "same_cat",
    histfunc = "count",
    title = "Same category in target vector (t1 = t2)",
    color_discrete_sequence = plt_style_s,

    width = 500,
    height = 500,
)

fig.add_hline(
    y = len(df_target.index.tolist()) / 2,
    line_width=3,
    line_dash="dash",
    line_color="grey",

    annotation_text = "q = 0.5",
    annotation_position="right",
    annotation_font_color = "black",
)

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
df_median

In [ ]:
fig = px.scatter(
    data_frame = df_median,
    x = df_median.index,
    y = "t2m_t1_mean",

    title = "Target: t2m_mean",
    color_continuous_scale = plt_style_c,
    labels = {"t2m_t1_mean" : "t2m_mean [k]"},

    color = "month",
)

scale_show(fig)

In [ ]:
if save_data is True:

    try:
        df.set_index(keys = "date", inplace = True)
    except:
        pass

    df.to_csv(os.path.join(data_folder, "df_main.csv"))

In [ ]:
#outlier analysis
df_median.iloc[59]

In [ ]:
df.loc[(df["day"].isin([27,28,29])) & (df["month"] == 2)][["t2m", "sp", "cdir"]].T

In [ ]:
#the outlier is neglecgtable

## 2.5 t2m and pv_breakdwon correlations

In [ ]:
df_comp = df[["date", "t2m", "pvs_latitude", "pvt_latitude", "pvt_pv_break_down_event", "pvs_pv_break_down_event", "t2m_t1_cat", "pvs_pv_edge", "pvt_pv_edge"]]
df_comp.head()

In [ ]:
# Correlation
df_corr = df_comp.corr().round(1)

# Mask to matrix
mask = np.zeros_like(df_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Viz
df_corr_viz = df_corr.mask(mask).dropna(how='all').dropna('columns', how='all')

fig = px.imshow(

    df_corr_viz,
    text_auto=True,
    color_continuous_scale = plt_style_c,

    title = "Correlation matrix",
    width = 700,
    height = 700,
    )

fig.show()

del df_corr

In [ ]:
fig = px.scatter(
    data_frame = df_comp,
    y = "t2m",
    x = "pvs_latitude",
    color = "pvs_pv_break_down_event",
    range_color = [0,1],
    opacity = 0.1,

    color_continuous_scale = plt_style_s,
    facet_col = "pvs_pv_edge",

    title = "t2m and pvs latitude correlation",
    labels = {"t2m" : "t2m [k]", "pvs_pv_break_down_event" : "breakdown"},
    trendline = "ols",
    trendline_color_override = "red",

)

fig.add_hline(
    y = 273.15,
    line_width=3,
    line_dash="dash",
    line_color="dark blue",

    annotation_text = "273.15",
    annotation_position="bottom left",
    annotation_font_color = "black",
)

scale_show(fig)

In [ ]:
fig = px.box(
    data_frame = df_comp,
    y = "t2m",
    x = "pvs_latitude",

    title = "t2m and pvs latitude correlation",
    labels = {"t2m" : "t2m ░k"},
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df_comp,
    y = "t2m",
    x = "pvt_latitude",
    color = "pvt_pv_break_down_event",
        range_color = [0,1],
    opacity = 0.1,

    color_continuous_scale = plt_style_s,
    facet_col = "pvt_pv_edge",

    title = "t2m and pvt latitude correlation",
    labels = {"t2m" : "t2m [k]", "pvt_pv_break_down_event" : "breakdown"},
    trendline = "ols",
    trendline_color_override = "red",
)

fig.add_hline(
    y = 273.15,
    line_width=3,
    line_dash="dash",
    line_color="dark blue",

    annotation_text = "273.15",
    annotation_position="bottom left",
    annotation_font_color = "black",
)


scale_show(fig)

In [ ]:
n = 7000

pvs_breakdown_list = df_comp.iloc[-n:].loc[df_comp["pvs_pv_break_down_event"] == 1]["date"].to_list()

fig = px.scatter(
    data_frame = df_comp.iloc[-n:],
    x = "date",
    y = "t2m",
    color = "t2m_t1_cat",
    color_continuous_scale = plt_style_s,
    title = "PVS break down events",
)

for date in pvs_breakdown_list:
    fig.add_vline(
        x = date,

    )


scale_show(fig)

In [ ]:
n = 7000

pvt_breakdown_list = df_comp.iloc[-n:].loc[df_comp["pvt_pv_break_down_event"] == 1]["date"].to_list()

fig = px.scatter(
    data_frame = df_comp.iloc[-n:],
    x = "date",
    y = "t2m",
    color = "t2m_t1_cat",
    color_continuous_scale = plt_style_s,
    title = "PVT break down events",
)

for date in pvt_breakdown_list:
    fig.add_vline(
        x = date,
    )


scale_show(fig)

In [ ]:
#comparison with ssw data
#source: https://csl.noaa.gov/groups/csl8/sswcompendium/majorevents.html

ssw_events = [
    "1958-01-01","1958-11-01","1960-01-01","1963-01-01","1965-03-01","1965-12-01","1966-02-01","1968-01-01","1968-11-01","1969-03-01","1970-01-01","1971-01-01","1971-03-01","1973-01-01","1977-01-01","1979-02-01","1980-02-01","1981-02-01","1981-03-01","1981-12-01","1984-02-01","1985-01-01","1987-01-01","1987-12-01","1988-03-01","1989-02-01","1998-12-01","1999-02-01","2000-03-01","2001-02-01","2001-12-01","2002-02-01","2003-01-01","2004-01-01","2006-01-01","2007-02-01","2008-02-01","2009-01-01","2010-02-01","2010-03-01","2013-01-01","2018-02-01","2019-01-01",
]


df_comp["ssw"] = 0
df_comp.loc[df_comp["date"].isin(ssw_events), "ssw"] = 1

In [ ]:
n = 5000
pvs_breakdown_list = df_comp.iloc[-n:].loc[df_comp["ssw"] == 1]["date"].to_list()

for col in ["ssw", "pvt_pv_break_down_event", "pvs_pv_break_down_event"]:

    bd_list = df_comp.iloc[-n:].loc[df_comp[col] == 1]["date"].to_list()


    fig = px.scatter(
        data_frame = df_comp.iloc[-n:],
        x = "date",
        y = df_comp.iloc[-n:]["t2m_t1_cat"].rolling(60).mean(),
        color = "t2m_t1_cat",
        color_continuous_scale =  plt_style_s,
        title = f"Break down event: {col}",

        labels = {"t2m" : "t2m [k]"}
    )

    for date in bd_list:
        fig.add_vline(
            x = date,
    )


    scale_show(fig)

In [ ]:
df_comp

In [ ]:
fig = px.line(
    data_frame = df_comp,
    x = "date",
    y = ["pvs_pv_break_down_event", "ssw"],

    title = "Polar vortex break down event (pvs) and SSW comparison",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df_comp,
    x = "date",
    y = ["pvt_pv_break_down_event","pvs_pv_break_down_event" ,"ssw"],

    title = "Polar vortex break down event (pvt, pvs) and SSW comparison",
    color_discrete_sequence = plt_style_s,

    labels = {"value" : "event", "pvt_pv_break_down_event" : "pvt","pvs_pv_break_down_event" : "pvs"},
)

scale_show(fig)

In [ ]:
df_comp[["pvt_pv_break_down_event", "pvs_pv_break_down_event", "ssw"]].sum()

In [ ]:
#del df_comp

# 3. Modeling - Random Forest

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import csv
import itertools

#folders
data_folder = "data"

#machine learning
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import RandomForestRegressor as rfr

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit as tsp
from sklearn.model_selection import GridSearchCV as gscv

#warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
run_optim = False #runtime: n min

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig, width = 1500, height = 750):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=width,
        height=height,
    )

    #show
    fig.show()

    return

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_main.csv"))
df.head().T

In [ ]:
#sources for reasoning

#data splitting: https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37

#data standardizing (train, test, valid):
#   https://stats.stackexchange.com/questions/202287/why-standardization-of-the-testing-set-has-to-be-performed-with-the-mean-and-sd
#   https://medium.com/analytics-vidhya/why-it-makes-a-difference-how-to-standardize-training-and-test-set-e95bf350bed3
#   https://stats.stackexchange.com/questions/248543/standardize-training-and-validation-data
#   https://www.kaggle.com/questions-and-answers/159183

#formula for standardizing: https://www.statisticshowto.com/standardized-values-examples/


In [ ]:
class Base(): #parent

    def __init__(self, df : object, y_col : list, data_folder : str, results_file : str, model_metric : str, n_jobs : int = 1, window : int = 30):

        #save raw df
        self.df_raw         = df.copy()

        #drop forbidden cols
        df = df.copy() #pass by value
        df = Base.__drop_forbidden_cols(df, y_col)

        #set dataframe for refferencing
        self.df             = df.copy() #windowed df, copy because obj is passed by refference

        #get and get x_col and y_col
        self.y_col          = y_col
        self.x_col          = list(df.drop(labels = y_col, axis = 1, inplace = False).columns.to_list())

        #misc params
        self.random_state   = 42
        self.n_jobs         = n_jobs
        self.data_folder    = data_folder
        self.results_file   = os.path.join(data_folder,results_file)
        self.model_metric   = model_metric

        #windowing parameters
        self.x_window       = window #number of shifting window input features

        self.__setup()

        return

    def __setup(self):

        #order was chosen to minimize data loss, at the cost of more needed processing power

        #data preparation
        self.__windowing()
        self.__split_data()
        self.__standardize_data()

        #setup of metrics and results
        self.__set_assesment()

        return

    @staticmethod
    def __drop_forbidden_cols(df, y_col):

        forbidden_cols = ['date','t2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', 't2m_t1_cat', 't2m_t2_cat']

        #prevent y_cols from being dropped from the data frame
        for y in y_col:
            if y in forbidden_cols:
                forbidden_cols.remove(y)

        #drop forbidden cols, to prevent adding future information to the time series
        print(f"Removed forbidden cols:\n{forbidden_cols}")
        df.drop(labels = forbidden_cols, axis = 1, inplace = True)

        return df

    def __windowing(self):
        """creates the windowed data frame"""

        self.x_col_windowed = self.x_col.copy() #copy, becaus lists are past by refference

        #input fetures: x
        for i in range(1, self.x_window + 1):
            for x_col in self.x_col: #inefficient but works just fine

                x_col_i             = f"{x_col}_-{i}"
                self.df[x_col_i]    = df[x_col].shift(i)

                self.x_col_windowed.append(x_col_i)

        #clean na columns, which were caused by the shifts
        self.df.dropna(inplace = True)
        print(f"\nApplying shifitng window:\nx_window: -{self.x_window}")

        return

    def __split_data(self):

        #reset index for splitting data
        self.df.reset_index(inplace = True, drop = True)
        length = self.df.shape[0]

        #setting split value fractions
        valid_frac_0      = 0.1
        test_frac_1       = 0.05
        train_frac_2      = 0.7
        valid_frac_3      = 0.1
        test_frac_4       = 0.05

        #get end indexes
        index_end_list = []
        cum_frac = 0

        for frac in [valid_frac_0, test_frac_1, train_frac_2, valid_frac_3, test_frac_4]:
            cum_frac += frac
            index_end_list.append(round(length * cum_frac))

        #get indexes (ugly code)
        df_indexes = self.df.index.tolist()
        train_i     = df_indexes[index_end_list[1] : index_end_list[2]]
        valid_i     = df_indexes[ : index_end_list[0]]                      + df_indexes[index_end_list[2] : index_end_list[3]]
        test_i      = df_indexes[index_end_list[0] : index_end_list[1]]     + df_indexes[index_end_list[3] : index_end_list[4]]

        #get df from indexes
        self.df_train_x = self.df[self.x_col_windowed].loc[self.df.index.isin(train_i)]
        self.df_train_y = self.df[self.y_col].loc[self.df.index.isin(train_i)]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].loc[self.df.index.isin(valid_i)]
        self.df_valid_y = self.df[self.y_col].loc[self.df.index.isin(valid_i)]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].loc[self.df.index.isin(test_i)]
        self.df_test_y = self.df[self.y_col].loc[self.df.index.isin(test_i)]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        #set data for plotting in raw df
        self.df_raw["set"] = None
        for index_items, set_type in zip([train_i, valid_i, test_i],["train", "valid", "test"]):
            self.df_raw.loc[self.df_raw.index.isin(index_items), "set"] = set_type

        return

    def plot_set_distribution(self, plotter, style, plt_style):

        if (style == "histogram") and (self.model_metric == "c"): #only plotlable with classificaiton model

            fig = px.histogram(
                data_frame = self.df_raw,
                x = "set",
                color = "t2m_t2_cat",
                histfunc = "count",

                barmode = "group",
                title = "Categorical distribution of sets",
                color_discrete_sequence = plt_style,
            )

        elif (style == "scatter"):

            fig = px.scatter(
                data_frame = self.df_raw,
                x = "date",
                y = "t2m",
                color = "set",

                title = "Trend distribution of sets",
                color_discrete_sequence = plt_style,
            )

        plotter(fig)

    def __split_data_deprecated(self):

        #df length
        length = self.df.shape[0]

        #setting split values
        valid_frac     = 0.2
        test_frac      = 0.1

        #get indexes
        train_end       = round(length * (1 - (valid_frac + test_frac)))
        valid_end       = round(length * (1 - (test_frac)))
        test_end        = round(length * (1))

        #create train df
        self.df_train_x = self.df[self.x_col_windowed].iloc[:train_end]
        self.df_train_y = self.df[self.y_col].iloc[:train_end]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].iloc[train_end:valid_end]
        self.df_valid_y = self.df[self.y_col].iloc[train_end:valid_end]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].iloc[valid_end:test_end]
        self.df_test_y = self.df[self.y_col].iloc[valid_end:test_end]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        return

    def __standardize_data(self):

        label_cat = [0,1]; label_cat.sort()
        self.standardizing_values = {
            "x" : {},
            "y" : {},
            #    col1 : {"mean" : value, "std"  : value},
            #    col2 : {"mean" : value, "std"  : value},
            #}
            #"y" : ...
        }

        print("\nStandardizing values:")
        for col in self.df.columns:

            distinct_values = list(self.df[col].unique())
            distinct_values.sort()

            if label_cat == distinct_values: #skip categorical values
                continue

            #get mean and std for all columns across both data both data frames
            if col in self.x_col_windowed:

                self.standardizing_values["x"][col]             = {}
                self.standardizing_values["x"][col]["mean"]     = self.df_train_x[col].mean()
                self.standardizing_values["x"][col]["std"]      = self.df_train_x[col].std()

            elif col in self.y_col:

                self.standardizing_values["y"][col]             = {}
                self.standardizing_values["y"][col]["mean"]     = self.df_train_y[col].mean()
                self.standardizing_values["y"][col]["std"]      = self.df_train_y[col].std()

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = (df[col] - mean) / std #standardization

        #check sum
        print(f"Checksum train x: {self.df_train_x[list(self.standardizing_values['x'].keys())].mean().round(2).sum()}")
        print(f"Checksum train y: {self.df_train_y[list(self.standardizing_values['y'].keys())].mean().round(2).sum()}")

        return

    def unstandardize_data(self):

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = df[col] * std + mean #reversed standardization

        return

    def __unstanardize_y(self, y_t1, y_t2):

        mean_t1        = self.standardizing_values["y"]["t2m_t1"]["mean"]
        mean_t2        = self.standardizing_values["y"]["t2m_t2"]["mean"]

        std_t1        = self.standardizing_values["y"]["t2m_t1"]["std"]
        std_t2        = self.standardizing_values["y"]["t2m_t2"]["std"]

        y_t1_unst  = y_t1 * std_t1 + mean_t1
        y_t2_unst  = y_t2 * std_t1 + mean_t2

        return y_t1_unst, y_t2_unst

    def __set_assesment(self):

        if self.model_metric == "c":
            self.get_model_score = self.__get_model_score_c

        elif self.model_metric == "r":
            self.get_model_score = self.__get_model_score_r

    def __get_model_score_c(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #default
        get_conf_mat = False
        mat_labels = [0,1]

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            get_conf_mat = True #only get confuciton matrix when a single model is created
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get acc
            score[f"{raw_key}_accuracy_t1"]     = round(accuracy_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_accuracy_t2"]     = round(accuracy_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_accuracy"]        = round(accuracy_score(y_true = y, y_pred = y_pred),3)

            #get conf mat
            if get_conf_mat is True:
                score[f"{raw_key}_mat_t1"]     = confusion_matrix(y_true = y_t1, y_pred = y_pred_t1, labels = mat_labels)
                score[f"{raw_key}_mat_t2"]     = confusion_matrix(y_true = y_t2, y_pred = y_pred_t2, labels = mat_labels)
                #score[f"{raw_key}_mat"]        = confusion_matrix(y_true = y, y_pred = y_pred) #multi labels are not supported

        #return metrics
        if get_conf_mat is True:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return score

        return score

    def __get_model_score_r(self, model = None, get_test_score = False, unstandardize_score = False): #used, when model_metric == "c"

        #if a model is passed, the function is calles from run_optim, otherwise,
        set_score = False

        if model is None: #model is not none when automation is run
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #unstandardize data (ugly code go brrrr)
            if unstandardize_score:
                y_t1, y_t2              = self.__unstanardize_y(y_t1 = y_t1, y_t2 = y_t2)
                y_pred_t1, y_pred_t2    = self.__unstanardize_y(y_t1 = y_pred_t1, y_t2 = y_pred_t2)

                y_pred[:,0], y_pred[:,1]                = y_pred_t1, y_pred_t2
                y[self.y_col[0]], y[self.y_col[1]]      = y_t1, y_t2

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def __get_model_score_r_deprecated(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def plot_confusion_mat(self, set = "valid"):
        """set = 'train', 'valid', 'test'"""

        mat_keys = [key for key in self.score.keys() if ("mat" in key) and (set in key)]

        for mat_key in mat_keys:

            mat = self.score[mat_key]
            title = str(mat_key).replace("_mat_", " ")

            fig  = px.imshow(
                mat,
                color_continuous_scale = px.colors.sequential.haline_r,
                text_auto = True,
            )

            #labels and layout
            fig.update_layout(

                title = f"Confusion matrix: {title}",

                width=500,
                height=500,

                xaxis_title="Predicted label",
                yaxis_title="True label",

                xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"]
                ),

                yaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"],
                ),
            )

            #set font
            fig.update_layout(
                font = dict(size=16),
                title_font = dict(size=20),
                xaxis_title_font = dict(size=18),
                yaxis_title_font = dict(size=18),
            )

            fig.show()

    def save_result(self, param, score):

        #merge and create a dataframe
        param.update(score); data = param
        df_result = pd.DataFrame([data])

        #create results file and set header length as param to negate reading file
        if os.path.isfile(self.results_file) is True:
            df_saved_result = pd.read_csv(self.results_file)
            df_result = df_saved_result.append(df_result)

        df_result.to_csv(self.results_file, index = False)

        return

    def get_results(self):

        df = pd.read_csv(self.results_file)
        return df

In [ ]:
class RF(Base): #child

    def run_optim(self, n_trees = 12, n_depth=3, n_leafs = 5):

        self.model = None #clear any models if there should be one

        param_list = self.create_param_list(
            n_trees = n_trees,
            n_depth = n_depth,
            n_leafs = n_leafs,
        )

        for param in param_list:

            print( f"Progress of optim:\t{round((param_list.index(param) / len(param_list)) * 100,1)}",end = "\r")

            score = self.create_model(param = param, single_model = False)
            self.save_result(param = param, score = score)

        print("Optim successfull. Read results with self.get_results()")
        return

    def create_param_list(self, n_trees, n_depth, n_leafs):
        """return list: [{param_1 : value_1},{},]"""

        param_list = []

        #get individual numbers
        n_estimators        = [2**n for n in range(1,n_trees+1)]
        max_depths          = [10*n for n in range(1,n_depth+1)]
        min_sample_leafs    = [2*n  for n in range(1,n_leafs+1)]

        combinations = list(itertools.product(n_estimators, max_depths, min_sample_leafs))

        for combination in combinations:

            params = {
                "n_estimators"      : combination[0],
                "max_depth"         : combination[1],
                "min_samples_leaf"  : combination[2],
            }

            param_list.append(params)

        print(f"\nGenerated param combinations: {len(param_list)}")
        return param_list


    def create_model(self, param, single_model = True):
        """if single_model == False:
            the scores get retuned
            self.mode is not set
        elif single_model == True:
            scores do not get returned
            seld.model is set"""

        #create model
        if self.model_metric == "r":
            ml_model = rfr

        elif self.model_metric == "c":
            ml_model = rfc

        model = ml_model(
            n_jobs          = self.n_jobs,
            random_state    = self.random_state,
            **param, #unpack the dict and dumps its values
        )

        #fit model
        model.fit(X = self.df_train_x, y = self.df_train_y)

        #set according metrics
        if single_model is True:
            self.model = model
            print(self.model)
            return

        elif single_model is False:
            score = self.get_model_score(model)
            return score

## 3.1 RF Calssification

### 3.1.1 Modeling and hyper parameter tuning

In [ ]:
rfc_obj = RF(
    df              = df,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 4,

    data_folder     = data_folder,
    results_file    = "optim_reults_rfc.csv",

    model_metric    =  "c" # r = regression, c = classification
)

In [ ]:
plot_dist = False

if plot_dist:
    rfc_obj.plot_set_distribution(plotter = scale_show, style = "histogram", plt_style = plt_style_s)
    rfc_obj.plot_set_distribution(plotter = scale_show, style = "scatter", plt_style = plt_style_s)

In [ ]:
if run_optim is True:
    rfc_obj.run_optim()

In [ ]:
df_results = rfc_obj.get_results()
df_results.sort_values(by = "valid_accuracy", ascending = False, inplace = True)
df_results.head(3)

In [ ]:
df_results.sort_values(by = "valid_accuracy_t1", ascending = False, inplace = True)
df_results.head(3)

In [ ]:
df_results.sort_values(by = "valid_accuracy_t2", ascending = False, inplace = True)
df_results.head(3)

In [ ]:
for frame in ["", "_t1", "_t2"]:

    title = "Fitting graph: estimators"

    if frame != "":
        title = f"Fitting graph {frame[1:]}: estimators"

    fig = px.scatter(

        data_frame = df_results,
        x = "n_estimators",
        y = [f"train_accuracy{frame}", f"valid_accuracy{frame}"],

        color_discrete_sequence = plt_style_s,
        title = title,
        log_x  = True,

        labels = {"value": "accuracy"},
        range_y = [0,1.1]

    )

    scale_show(fig)

### 3.1.2 Top model evaluation

In [ ]:
#creating sinlge model
optimal_param = {
    "n_estimators"      : 128,
    "max_depth"         : 10,
    "min_samples_leaf"   : 8
}

rfc_obj.create_model(param = optimal_param)
rfc_obj.get_model_score()

https://stackoverflow.com/questions/60860121/plotly-how-to-make-an-annotated-confusion-matrix-using-a-heatmap

In [ ]:
rfc_obj.plot_confusion_mat(set = "valid")

In [ ]:
# confusion mat
#add plotting to class

In [ ]:
# feature importance and further improvment
weights         = rfc_obj.model.feature_importances_
cols_window     = rfc_obj.x_col_windowed
n_col           = len(rfc_obj.x_col)
window_size     = rfc_obj.x_window

mat_head    = rfc_obj.x_col
mat         = []

last_satrt = 0

for i in range(1, 1+window_size):

    end = i * n_col
    mat.append(list(weights[last_satrt:end]))

    last_satrt = end

df_feature_importance = pd.DataFrame(mat, columns = rfc_obj.x_col)

In [ ]:
df_feature_importance_by_f = df_feature_importance.sum()
df_feature_importance_by_f.sort_values(inplace = True, ascending = False)

fig = px.histogram(
    x = df_feature_importance_by_f,
    y = df_feature_importance_by_f.index,
    nbins = n_col,
    histfunc = "sum",
    color_discrete_sequence = plt_style_s,
    title = "Feature importance by feature",

    labels = {"x" : "weights", "y" : "feature"}
)

fig.update_yaxes(autorange="reversed")

scale_show(fig, width = 750, height = 750)

In [ ]:
df_feature_importance_by_t = df_feature_importance.T.sum()

In [ ]:
df_feature_importance_by_t = df_feature_importance.T.sum()
#df_feature_importance_by_t.sort_values(inplace = True, ascending = False)

fig = px.histogram(
    x = df_feature_importance_by_t,
    y = df_feature_importance_by_t.index.astype(str),
    nbins = n_col,
    histfunc = "sum",
    color_discrete_sequence = plt_style_s,
    title = "Feature importance by time",

    labels = {"x" : "weights", "y" : "time offset"}
)

fig.update_yaxes(autorange="reversed")

scale_show(fig, width = 750, height = 750)

In [ ]:
fig = px.imshow(df_feature_importance.T.to_numpy(), color_continuous_scale = plt_style_c)

fig.update_layout(

    title = f"Feature importance",

    width=1000,
    height=1000,

    yaxis_title="Feature",
    xaxis_title="Time",

    yaxis = dict(
        tickmode = 'array',
        tickvals = list(range(len(rfc_obj.x_col))),
        ticktext =  rfc_obj.x_col,
    ),
)

fig.update_layout(
    font = dict(size=16),
    title_font = dict(size=20),
    xaxis_title_font = dict(size=18),
    yaxis_title_font = dict(size=18),
)

fig.show()

In [ ]:
#apply to testing set

In [ ]:
rfc_obj.get_model_score( get_test_score = True)

In [ ]:
rfc_obj.plot_confusion_mat(set = "test")

## 3.1.3 Exploration of missing features

In [ ]:
df_mis = df.drop(labels = "year", axis = 1, inplace = False)

In [ ]:
rfc_obj_mis = RF(
    df              = df_mis,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 4,

    data_folder     = data_folder,
    results_file    = "optim_reults_rfc.csv",

    model_metric    =  "c" # r = regression, c = classification
)

In [ ]:
rfc_obj_mis.create_model(param = optimal_param)
rfc_obj_mis.get_model_score(get_test_score = True)

In [ ]:
rfc_obj_mis.plot_confusion_mat(set = "test")

## 3.2 Dimensionality reduction

In [ ]:
#dropping feautres
set_0 = df_feature_importance_by_f.index[:5].tolist() + ['date', 't2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', "t2m_t1_cat", "t2m_t2_cat"]
set_1 = df_feature_importance_by_f.index[:10].tolist() + ['date', 't2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', "t2m_t1_cat", "t2m_t2_cat"]
set_2 = df_feature_importance_by_f.index[:15].tolist() + ['date', 't2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', "t2m_t1_cat", "t2m_t2_cat"]
set_3 = df.drop(labels = ["nao", "ao", "t2m", "mjo_amplitude", "sp", "day", "soi"], axis = 1, inplace = False).columns

In [ ]:
df_0 = df[set_0]
df_1 = df[set_1]
df_2 = df[set_2]
df_3 = df[set_3]

In [ ]:
def magic(df, set_name):

    rfc_set = RF(
        df              = df,
        y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

        n_jobs          = 4,

        data_folder     = data_folder,
        results_file    = "optim_reults_rfc.csv",

        model_metric    =  "c" # r = regression, c = classification
    )

    #creating sinlge model
    optimal_param = {
            "n_estimators"      : 128,
            "max_depth"         : 10,
            "min_samples_leaf"   : 8
        }

    rfc_set.create_model(param = optimal_param)

    print(f"\n{set_name}")
    score = rfc_set.get_model_score(get_test_score = False)

    return score

In [ ]:
results = {}

for df_set, set_name in zip([df_0, df_1, df_2, df_3], ["top 5 features", "top 10 features", "top 15 features", "handpicked"]):

    result = magic(df_set, set_name)
    results[set_name] = result

In [ ]:
results

In [ ]:
df_result = pd.DataFrame(results)
df_result.drop(labels = [ind for ind in df_result.index if "mat" in ind], axis = 0, inplace = True)
df_result.T


## 3.3 Overfitting prevention

In [ ]:
def magic(df, param):

    rfc_param = RF(
        df              = df,
        y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

        n_jobs          = 4,

        data_folder     = data_folder,
        results_file    = "optim_reults_rfc.csv",

        model_metric    =  "c" # r = regression, c = classification
    )

    rfc_param.create_model(param = param)
    score = rfc_param.get_model_score(get_test_score = False)

    return score

In [ ]:
df.T

In [ ]:
#creating sinlge model
param_set0 = {
        "n_estimators"      : 100,
        "max_depth"         : 10,
        "min_samples_leaf"  : 8,
    }

#creating sinlge model
param_set1 = {
        "n_estimators"      : 128,
        "max_depth"         : 5,
        "min_samples_leaf"  : 8,
    }

#creating sinlge model
param_set2 = {
        "n_estimators"      : 128,
        "max_depth"         : 10,
        "min_samples_leaf"  : 200,
    }

#creating sinlge model
param_set3 = {
        "n_estimators"      : 128,
        "max_depth"         : 10,
        "min_samples_leaf"  : 8,
        "min_samples_split" : 200,
    }

results = []

for param in [param_set0, param_set1, param_set2, param_set3]:

    result = magic(df_2, param)
    result.update(param)
    results.append(result)

In [ ]:
df_results = pd.DataFrame(results)
df_results.T

In [ ]:
#overwrite RF param generator
class RFOP(RF):

    def create_param_list(self, n_trees, n_depth, n_leafs):
        """return list: [{param_1 : value_1},{},]"""

        param_list = []

        #get individual numbers
        n_estimators        = [128]
        max_depths          = [2*n for n in range(1,n_depth+1)]
        min_sample_leafs    = [25*n  for n in range(1,n_leafs+1)]

        combinations = list(itertools.product(n_estimators, max_depths, min_sample_leafs))

        for combination in combinations:

            params = {
                "n_estimators"      : combination[0],
                "max_depth"         : combination[1],
                "min_samples_leaf"  : combination[2],
            }

            param_list.append(params)

        print(f"\nGenerated param combinations: {len(param_list)}")
        return param_list

In [ ]:
rfc_op_obj = RFOP(
    df              = df,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 4,

    data_folder     = data_folder,
    results_file    = "optim_reults_rfc_op.csv",

    model_metric    =  "c" # r = regression, c = classification
)

In [ ]:
df

In [ ]:
if run_optim:
     rfc_op_obj.run_optim(n_depth = 7, n_leafs = 12)

In [ ]:
df_results_op = rfc_op_obj.get_results()
df_results_op.sort_values(by = "valid_accuracy", ascending = False, inplace = True)

df_results_op.head(10)

In [ ]:
fig = px.scatter(
    data_frame = df_results_op,
    x = "train_accuracy",
    y = "valid_accuracy",
    color = "min_samples_leaf",
    size = "max_depth",

    title = "Fitting graph RF Classifier",
    trendline = "lowess",
    color_continuous_scale = plt_style_c,
)

fig.add_hline(
    y = 0.55,
)
fig.add_vline(
    x = 0.5
)

fig.add_scatter()

scale_show(fig)

In [ ]:
optimal_param = {
    "n_estimators" : 128,
    "max_depth": 8,
    "min_samples_leaf": 225 #225
}

In [ ]:
#create optimal model
rfc_op_obj.create_model(param = optimal_param)
rfc_op_obj.get_model_score(get_test_score = True)
rfc_op_obj.plot_confusion_mat(set = "test")

In [ ]:
[item for item in df.columns.tolist() if item not in df_2.columns.tolist()]

In [ ]:
df_2.columns.tolist()

In [ ]:
rfc_op_obj = RFOP(
    df              = df_2,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 4,

    data_folder     = data_folder,
    results_file    = "optim_reults_rfc_op.csv",

    model_metric    =  "c" # r = regression, c = classification
)

In [ ]:
#create optimal model
rfc_op_obj.create_model(param = optimal_param)
rfc_op_obj.get_model_score(get_test_score = True)
rfc_op_obj.plot_confusion_mat(set = "test")

# 4. Modeling - Nu support vector machine

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import itertools

#folders
data_folder = "data"

#machine learning
from sklearn.svm import NuSVC as svc
from sklearn.svm import NuSVR as svr
from sklearn.multioutput import MultiOutputClassifier #nusvc does not support mutli calss natively
from sklearn.multioutput import MultiOutputRegressor #nusvc does not support mutli calss natively

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
run_optim = False #runtime: n min

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_main.csv"))
df.head().T

In [ ]:
class Base(): #parent

    def __init__(self, df : object, y_col : list, data_folder : str, results_file : str, model_metric : str, n_jobs : int = 1, window : int = 30):

        #save raw df
        self.df_raw         = df.copy()

        #drop forbidden cols
        df = df.copy() #pass by value
        df = Base.__drop_forbidden_cols(df, y_col)

        #set dataframe for refferencing
        self.df             = df.copy() #windowed df, copy because obj is passed by refference

        #get and get x_col and y_col
        self.y_col          = y_col
        self.x_col          = list(df.drop(labels = y_col, axis = 1, inplace = False).columns.to_list())

        #misc params
        self.random_state   = 42
        self.n_jobs         = n_jobs
        self.data_folder    = data_folder
        self.results_file   = os.path.join(data_folder,results_file)
        self.model_metric   = model_metric

        #windowing parameters
        self.x_window       = window #number of shifting window input features

        self.__setup()

        return

    def __setup(self):

        #order was chosen to minimize data loss, at the cost of more needed processing power

        #data preparation
        self.__windowing()
        self.__split_data()
        self.__standardize_data()

        #setup of metrics and results
        self.__set_assesment()

        return

    @staticmethod
    def __drop_forbidden_cols(df, y_col):

        forbidden_cols = ['date','t2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', 't2m_t1_cat', 't2m_t2_cat']

        #prevent y_cols from being dropped from the data frame
        for y in y_col:
            if y in forbidden_cols:
                forbidden_cols.remove(y)

        #drop forbidden cols, to prevent adding future information to the time series
        print(f"Removed forbidden cols:\n{forbidden_cols}")
        df.drop(labels = forbidden_cols, axis = 1, inplace = True)

        return df

    def __windowing(self):
        """creates the windowed data frame"""

        self.x_col_windowed = self.x_col.copy() #copy, becaus lists are past by refference

        #input fetures: x
        for i in range(1, self.x_window + 1):
            for x_col in self.x_col: #inefficient but works just fine

                x_col_i             = f"{x_col}_-{i}"
                self.df[x_col_i]    = df[x_col].shift(i)

                self.x_col_windowed.append(x_col_i)

        #clean na columns, which were caused by the shifts
        self.df.dropna(inplace = True)
        print(f"\nApplying shifitng window:\nx_window: -{self.x_window}")

        return

    def __split_data(self):

        #reset index for splitting data
        self.df.reset_index(inplace = True, drop = True)
        length = self.df.shape[0]

        #setting split value fractions
        valid_frac_0      = 0.1
        test_frac_1       = 0.05
        train_frac_2      = 0.7
        valid_frac_3      = 0.1
        test_frac_4       = 0.05

        #get end indexes
        index_end_list = []
        cum_frac = 0

        for frac in [valid_frac_0, test_frac_1, train_frac_2, valid_frac_3, test_frac_4]:
            cum_frac += frac
            index_end_list.append(round(length * cum_frac))

        #get indexes (ugly code)
        df_indexes = self.df.index.tolist()
        train_i     = df_indexes[index_end_list[1] : index_end_list[2]]
        valid_i     = df_indexes[ : index_end_list[0]]                      + df_indexes[index_end_list[2] : index_end_list[3]]
        test_i      = df_indexes[index_end_list[0] : index_end_list[1]]     + df_indexes[index_end_list[3] : index_end_list[4]]

        #get df from indexes
        self.df_train_x = self.df[self.x_col_windowed].loc[self.df.index.isin(train_i)]
        self.df_train_y = self.df[self.y_col].loc[self.df.index.isin(train_i)]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].loc[self.df.index.isin(valid_i)]
        self.df_valid_y = self.df[self.y_col].loc[self.df.index.isin(valid_i)]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].loc[self.df.index.isin(test_i)]
        self.df_test_y = self.df[self.y_col].loc[self.df.index.isin(test_i)]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        #set data for plotting in raw df
        self.df_raw["set"] = None
        for index_items, set_type in zip([train_i, valid_i, test_i],["train", "valid", "test"]):
            self.df_raw.loc[self.df_raw.index.isin(index_items), "set"] = set_type

        return

    def plot_set_distribution(self, plotter, style, plt_style):

        if (style == "histogram") and (self.model_metric == "c"): #only plotlable with classificaiton model

            fig = px.histogram(
                data_frame = self.df_raw,
                x = "set",
                color = "t2m_t2_cat",
                histfunc = "count",

                barmode = "group",
                title = "Categorical distribution of sets",
                color_discrete_sequence = plt_style,
            )

        elif (style == "scatter"):

            fig = px.scatter(
                data_frame = self.df_raw,
                x = "date",
                y = "t2m",
                color = "set",

                title = "Trend distribution of sets",
                color_discrete_sequence = plt_style,
            )

        plotter(fig)

    def __split_data_deprecated(self):

        #df length
        length = self.df.shape[0]

        #setting split values
        valid_frac     = 0.2
        test_frac      = 0.1

        #get indexes
        train_end       = round(length * (1 - (valid_frac + test_frac)))
        valid_end       = round(length * (1 - (test_frac)))
        test_end        = round(length * (1))

        #create train df
        self.df_train_x = self.df[self.x_col_windowed].iloc[:train_end]
        self.df_train_y = self.df[self.y_col].iloc[:train_end]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].iloc[train_end:valid_end]
        self.df_valid_y = self.df[self.y_col].iloc[train_end:valid_end]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].iloc[valid_end:test_end]
        self.df_test_y = self.df[self.y_col].iloc[valid_end:test_end]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        return

    def __standardize_data(self):

        label_cat = [0,1]; label_cat.sort()
        self.standardizing_values = {
            "x" : {},
            "y" : {},
            #    col1 : {"mean" : value, "std"  : value},
            #    col2 : {"mean" : value, "std"  : value},
            #}
            #"y" : ...
        }

        print("\nStandardizing values:")
        for col in self.df.columns:

            distinct_values = list(self.df[col].unique())
            distinct_values.sort()

            if label_cat == distinct_values: #skip categorical values
                continue

            #get mean and std for all columns across both data both data frames
            if col in self.x_col_windowed:

                self.standardizing_values["x"][col]             = {}
                self.standardizing_values["x"][col]["mean"]     = self.df_train_x[col].mean()
                self.standardizing_values["x"][col]["std"]      = self.df_train_x[col].std()

            elif col in self.y_col:

                self.standardizing_values["y"][col]             = {}
                self.standardizing_values["y"][col]["mean"]     = self.df_train_y[col].mean()
                self.standardizing_values["y"][col]["std"]      = self.df_train_y[col].std()

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = (df[col] - mean) / std #standardization

        #check sum
        print(f"Checksum train x: {self.df_train_x[list(self.standardizing_values['x'].keys())].mean().round(2).sum()}")
        print(f"Checksum train y: {self.df_train_y[list(self.standardizing_values['y'].keys())].mean().round(2).sum()}")

        return

    def unstandardize_data(self):

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = df[col] * std + mean #reversed standardization

        return

    def __unstanardize_y(self, y_t1, y_t2):

        mean_t1        = self.standardizing_values["y"]["t2m_t1"]["mean"]
        mean_t2        = self.standardizing_values["y"]["t2m_t2"]["mean"]

        std_t1        = self.standardizing_values["y"]["t2m_t1"]["std"]
        std_t2        = self.standardizing_values["y"]["t2m_t2"]["std"]

        y_t1_unst  = y_t1 * std_t1 + mean_t1
        y_t2_unst  = y_t2 * std_t1 + mean_t2

        return y_t1_unst, y_t2_unst

    def __set_assesment(self):

        if self.model_metric == "c":
            self.get_model_score = self.__get_model_score_c

        elif self.model_metric == "r":
            self.get_model_score = self.__get_model_score_r

    def __set_assesment(self):

        if self.model_metric == "c":
            self.get_model_score = self.__get_model_score_c

        elif self.model_metric == "r":
            self.get_model_score = self.__get_model_score_r

    def __get_model_score_c(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #default
        get_conf_mat = False
        mat_labels = [0,1]

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            get_conf_mat = True #only get confuciton matrix when a single model is created
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get acc
            score[f"{raw_key}_accuracy_t1"]     = round(accuracy_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_accuracy_t2"]     = round(accuracy_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_accuracy"]        = round(accuracy_score(y_true = y, y_pred = y_pred),3)

            #get conf mat
            if get_conf_mat is True:
                score[f"{raw_key}_mat_t1"]     = confusion_matrix(y_true = y_t1, y_pred = y_pred_t1, labels = mat_labels)
                score[f"{raw_key}_mat_t2"]     = confusion_matrix(y_true = y_t2, y_pred = y_pred_t2, labels = mat_labels)
                #score[f"{raw_key}_mat"]        = confusion_matrix(y_true = y, y_pred = y_pred) #multi labels are not supported

        #return metrics
        if get_conf_mat is True:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def __get_model_score_r(self, model = None, get_test_score = False, unstandardize_score = False): #used, when model_metric == "c"

        #if a model is passed, the function is calles from run_optim, otherwise,
        set_score = False

        if model is None: #model is not none when automation is run
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #unstandardize data (ugly code go brrrr)
            if unstandardize_score:
                y_t1, y_t2              = self.__unstanardize_y(y_t1 = y_t1, y_t2 = y_t2)
                y_pred_t1, y_pred_t2    = self.__unstanardize_y(y_t1 = y_pred_t1, y_t2 = y_pred_t2)

                y_pred[:,0], y_pred[:,1]                = y_pred_t1, y_pred_t2
                y[self.y_col[0]], y[self.y_col[1]]      = y_t1, y_t2

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def __get_model_score_r_deprecated(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def plot_confusion_mat(self, set = "valid"):
        """set = 'train', 'valid', 'test'"""

        mat_keys = [key for key in self.score.keys() if ("mat" in key) and (set in key)]

        for mat_key in mat_keys:

            mat = self.score[mat_key]
            title = str(mat_key).replace("_mat_", " ")

            fig  = px.imshow(
                mat,
                color_continuous_scale = px.colors.sequential.haline_r,
                text_auto = True,
            )

            #labels and layout
            fig.update_layout(

                title = f"Confusion matrix: {title}",

                width=500,
                height=500,

                xaxis_title="Predicted label",
                yaxis_title="True label",

                xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"]
                ),

                yaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"],
                ),
            )

            #set font
            fig.update_layout(
                font = dict(size=16),
                title_font = dict(size=20),
                xaxis_title_font = dict(size=18),
                yaxis_title_font = dict(size=18),
            )

            fig.show()

    def save_result(self, param, score):

        #merge and create a dataframe
        param.update(score); data = param
        df_result = pd.DataFrame([data])

        #create results file and set header length as param to negate reading file
        if os.path.isfile(self.results_file) is True:
            df_saved_result = pd.read_csv(self.results_file)
            df_result = df_saved_result.append(df_result)

        df_result.to_csv(self.results_file, index = False)

        return

    def get_results(self):

        df = pd.read_csv(self.results_file)
        return df

In [ ]:
class SV(Base): #child

    def run_optim(self, nu_max = 11, degrees = 9):

        self.model = None #clear any models if there should be one

        param_list = self.create_param_list(
            nu_max = nu_max,
            degrees = degrees,
        )

        for param in param_list:

            print( f"Progress of optim:\t{round((param_list.index(param) / len(param_list)) * 100,1)}",end = "\r")

            score = self.create_model(param = param, single_model = False)
            self.save_result(param = param, score = score)
            break

        print("Optim successfull. Read results with self.get_results()")
        return

    def create_param_list(self, nu_max, degrees):
        """return list: [{param_1 : value_1},{},]"""

        param_list = []

        #set values to optimze for with the model
        kernels     = ["linear", "rbf", "sigmoid", "poly"]
        nus         = [i / 10 for i in range(1,nu_max)]
        degrees     = list(range(1,degrees))
        gammas       = ["scale", "auto", 0.1, 0.5, 1, 5, 10]

        #create kernel dicts
        combinations = itertools.product(kernels, nus, degrees, gammas)

        for combination in combinations:

            #kick out unvalid combiations to minimize calculation time
            if (combination[0] != "poly") and (combination[2] != 1):
                continue
            if (combination[0] == "linear") and (combination[3] != "scale"): #scale is the default param
                continue

            params = {
                "kernel"        : combination[0],
                "nu"            : combination[1],
                "degree"        : combination[2],
                "gamma"         : combination[3],
            }

            param_list.append(params)

        print(f"\nGenerated param combinations: {len(param_list)}")
        return param_list

    def create_model(self, param, single_model = True):
        """if single_model == False:
            the scores get retuned
            self.mode is not set
        elif single_model == True:
            scores do not get returned
            seld.model is set"""

        #create model
        if self.model_metric == "r":
            ml_model = svr
            multi_output = MultiOutputRegressor

        elif self.model_metric == "c":
            ml_model = svc
            multi_output = MultiOutputClassifier

        model = multi_output(
            n_jobs = self.n_jobs,
            estimator = ml_model(
                random_state    = self.random_state,
                **param, #unpack the dict and dumps its values
            )
        )

        #fit model
        model.fit(X = self.df_train_x, Y = self.df_train_y)

        #set according metrics
        if single_model is True:
            self.model = model
            print(self.model)
            return

        elif single_model is False:
            score = self.get_model_score(model)
            return score

In [ ]:
svc_obj = SV(
    df              = df,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 5,

    data_folder     = data_folder,
    results_file    = "optim_reults_svc.csv",

    model_metric    =  "c", # r = regression, c = classification
    window          = 2,
)

In [ ]:
if run_optim is True:
    svc_obj.run_optim()

## 4.2 Model evaluation:
- Too high dimensionality
- Too many datapoints
- Calculation times for a singler model with the given paramets are way too high

# 5. Modeling - Multi layer preceptron

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#folders
data_folder = "data"

#machine learning
from sklearn.neural_network import MLPClassifier    as mlc
from sklearn.neural_network import MLPRegressor     as mlr

#model scoring
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
run_optim = False #runtime: 711 min

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_main.csv"))
df.head().T

## 5.1 Data preparation, Modeling, Architecutre tuning

In [ ]:
class Base(): #parent

    def __init__(self, df : object, y_col : list, data_folder : str, results_file : str, model_metric : str, n_jobs : int = 1, window : int = 30):

        #save raw df
        self.df_raw         = df.copy()

        #drop forbidden cols
        df = df.copy() #pass by value
        df = Base.__drop_forbidden_cols(df, y_col)

        #set dataframe for refferencing
        self.df             = df.copy() #windowed df, copy because obj is passed by refference

        #get and get x_col and y_col
        self.y_col          = y_col
        self.x_col          = list(df.drop(labels = y_col, axis = 1, inplace = False).columns.to_list())

        #misc params
        self.random_state   = 42
        self.n_jobs         = n_jobs
        self.data_folder    = data_folder
        self.results_file   = os.path.join(data_folder,results_file)
        self.model_metric   = model_metric

        #ann parameters
        self.default_param = {
            "activation"        : "relu",
            "solver"            : "adam",       #stochastic gradiant descent
            "alpha"             : 0.1,          #see: https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html
            "learning_rate"     : "adaptive",   #trying to improve performance
            "shuffle"           : False,        #keep order because of time series
            "early_stopping"    : True,         #trying to reduce processing time
            "max_iter"          : 200,          #change this if the training is too slow
        }

        #windowing parameters
        self.x_window       = window #number of shifting window input features

        self.__setup()

        return

    def __setup(self):

        #order was chosen to minimize data loss, at the cost of more needed processing power

        #data preparation
        self.__windowing()
        self.__split_data()
        self.__standardize_data()

        #setup of metrics and results
        self.__set_assesment()

        return

    @staticmethod
    def __drop_forbidden_cols(df, y_col):

        forbidden_cols = ['date','t2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', 't2m_t1_cat', 't2m_t2_cat']

        #prevent y_cols from being dropped from the data frame
        for y in y_col:
            if y in forbidden_cols:
                forbidden_cols.remove(y)

        #drop forbidden cols, to prevent adding future information to the time series
        print(f"Removed forbidden cols:\n{forbidden_cols}")
        df.drop(labels = forbidden_cols, axis = 1, inplace = True)

        return df

    def __windowing(self):
        """creates the windowed data frame"""

        self.x_col_windowed = self.x_col.copy() #copy, becaus lists are past by refference

        #input fetures: x
        for i in range(1, self.x_window + 1):
            for x_col in self.x_col: #inefficient but works just fine

                x_col_i             = f"{x_col}_-{i}"
                self.df[x_col_i]    = df[x_col].shift(i)

                self.x_col_windowed.append(x_col_i)

        #clean na columns, which were caused by the shifts
        self.df.dropna(inplace = True)
        print(f"\nApplying shifitng window:\nx_window: -{self.x_window}")

        return

    def __split_data(self):

        #reset index for splitting data
        self.df.reset_index(inplace = True, drop = True)
        length = self.df.shape[0]

        #setting split value fractions
        valid_frac_0      = 0.1
        test_frac_1       = 0.05
        train_frac_2      = 0.7
        valid_frac_3      = 0.1
        test_frac_4       = 0.05

        #get end indexes
        index_end_list = []
        cum_frac = 0

        for frac in [valid_frac_0, test_frac_1, train_frac_2, valid_frac_3, test_frac_4]:
            cum_frac += frac
            index_end_list.append(round(length * cum_frac))

        #get indexes (ugly code)
        df_indexes = self.df.index.tolist()
        train_i     = df_indexes[index_end_list[1] : index_end_list[2]]
        valid_i     = df_indexes[ : index_end_list[0]]                      + df_indexes[index_end_list[2] : index_end_list[3]]
        test_i      = df_indexes[index_end_list[0] : index_end_list[1]]     + df_indexes[index_end_list[3] : index_end_list[4]]

        #get df from indexes
        self.df_train_x = self.df[self.x_col_windowed].loc[self.df.index.isin(train_i)]
        self.df_train_y = self.df[self.y_col].loc[self.df.index.isin(train_i)]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].loc[self.df.index.isin(valid_i)]
        self.df_valid_y = self.df[self.y_col].loc[self.df.index.isin(valid_i)]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].loc[self.df.index.isin(test_i)]
        self.df_test_y = self.df[self.y_col].loc[self.df.index.isin(test_i)]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        #set data for plotting in raw df
        self.df_raw["set"] = None
        for index_items, set_type in zip([train_i, valid_i, test_i],["train", "valid", "test"]):
            self.df_raw.loc[self.df_raw.index.isin(index_items), "set"] = set_type

        return

    def plot_set_distribution(self, plotter, style, plt_style):

        if (style == "histogram") and (self.model_metric == "c"): #only plotlable with classificaiton model

            fig = px.histogram(
                data_frame = self.df_raw,
                x = "set",
                color = "t2m_t2_cat",
                histfunc = "count",

                barmode = "group",
                title = "Categorical distribution of sets",
                color_discrete_sequence = plt_style,
            )

        elif (style == "scatter"):

            fig = px.scatter(
                data_frame = self.df_raw,
                x = "date",
                y = "t2m",
                color = "set",

                title = "Trend distribution of sets",
                color_discrete_sequence = plt_style,
            )

        plotter(fig)

    def __split_data_deprecated(self):

        #df length
        length = self.df.shape[0]

        #setting split values
        valid_frac     = 0.2
        test_frac      = 0.1

        #get indexes
        train_end       = round(length * (1 - (valid_frac + test_frac)))
        valid_end       = round(length * (1 - (test_frac)))
        test_end        = round(length * (1))

        #create train df
        self.df_train_x = self.df[self.x_col_windowed].iloc[:train_end]
        self.df_train_y = self.df[self.y_col].iloc[:train_end]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].iloc[train_end:valid_end]
        self.df_valid_y = self.df[self.y_col].iloc[train_end:valid_end]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].iloc[valid_end:test_end]
        self.df_test_y = self.df[self.y_col].iloc[valid_end:test_end]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        return

    def __standardize_data(self):

        label_cat = [0,1]; label_cat.sort()
        self.standardizing_values = {
            "x" : {},
            "y" : {},
            #    col1 : {"mean" : value, "std"  : value},
            #    col2 : {"mean" : value, "std"  : value},
            #}
            #"y" : ...
        }

        print("\nStandardizing values:")
        for col in self.df.columns:

            distinct_values = list(self.df[col].unique())
            distinct_values.sort()

            if label_cat == distinct_values: #skip categorical values
                continue

            #get mean and std for all columns across both data both data frames
            if col in self.x_col_windowed:

                self.standardizing_values["x"][col]             = {}
                self.standardizing_values["x"][col]["mean"]     = self.df_train_x[col].mean()
                self.standardizing_values["x"][col]["std"]      = self.df_train_x[col].std()

            elif col in self.y_col:

                self.standardizing_values["y"][col]             = {}
                self.standardizing_values["y"][col]["mean"]     = self.df_train_y[col].mean()
                self.standardizing_values["y"][col]["std"]      = self.df_train_y[col].std()

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = (df[col] - mean) / std #standardization

        #check sum
        print(f"Checksum train x: {self.df_train_x[list(self.standardizing_values['x'].keys())].mean().round(2).sum()}")
        print(f"Checksum train y: {self.df_train_y[list(self.standardizing_values['y'].keys())].mean().round(2).sum()}")

        return

    def unstandardize_data(self):

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = df[col] * std + mean #reversed standardization

        return

    def __unstanardize_y(self, y_t1, y_t2):

        mean_t1        = self.standardizing_values["y"]["t2m_t1"]["mean"]
        mean_t2        = self.standardizing_values["y"]["t2m_t2"]["mean"]

        std_t1        = self.standardizing_values["y"]["t2m_t1"]["std"]
        std_t2        = self.standardizing_values["y"]["t2m_t2"]["std"]

        y_t1_unst  = y_t1 * std_t1 + mean_t1
        y_t2_unst  = y_t2 * std_t1 + mean_t2

        return y_t1_unst, y_t2_unst

    def __set_assesment(self):

        if self.model_metric == "c":
            self.get_model_score = self.__get_model_score_c

        elif self.model_metric == "r":
            self.get_model_score = self.__get_model_score_r

    def __get_model_score_c(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #default
        get_conf_mat = False
        mat_labels = [0,1]

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            get_conf_mat = True #only get confuciton matrix when a single model is created
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get acc
            score[f"{raw_key}_accuracy_t1"]     = round(accuracy_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_accuracy_t2"]     = round(accuracy_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_accuracy"]        = round(accuracy_score(y_true = y, y_pred = y_pred),3)

            #get conf mat
            if get_conf_mat is True:
                score[f"{raw_key}_mat_t1"]     = confusion_matrix(y_true = y_t1, y_pred = y_pred_t1, labels = mat_labels)
                score[f"{raw_key}_mat_t2"]     = confusion_matrix(y_true = y_t2, y_pred = y_pred_t2, labels = mat_labels)
                #score[f"{raw_key}_mat"]        = confusion_matrix(y_true = y, y_pred = y_pred) #multi labels are not supported

        #return metrics
        if get_conf_mat is True:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def __get_model_score_r(self, model = None, get_test_score = False, unstandardize_score = False): #used, when model_metric == "c"

        #if a model is passed, the function is calles from run_optim, otherwise,
        set_score = False

        if model is None: #model is not none when automation is run
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #unstandardize data (ugly code go brrrr)
            if unstandardize_score:
                y_t1, y_t2              = self.__unstanardize_y(y_t1 = y_t1, y_t2 = y_t2)
                y_pred_t1, y_pred_t2    = self.__unstanardize_y(y_t1 = y_pred_t1, y_t2 = y_pred_t2)

                y_pred[:,0], y_pred[:,1]                = y_pred_t1, y_pred_t2
                y[self.y_col[0]], y[self.y_col[1]]      = y_t1, y_t2

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def plot_confusion_mat(self, set = "valid"):
        """set = 'train', 'valid', 'test'"""

        mat_keys = [key for key in self.score.keys() if ("mat" in key) and (set in key)]

        for mat_key in mat_keys:

            mat = self.score[mat_key]
            title = str(mat_key).replace("_mat_", " ")

            fig  = px.imshow(
                mat,
                color_continuous_scale = px.colors.sequential.haline_r,
                text_auto = True,
            )

            #labels and layout
            fig.update_layout(

                title = f"Confusion matrix: {title}",

                width=500,
                height=500,

                xaxis_title="Predicted label",
                yaxis_title="True label",

                xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"]
                ),

                yaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"],
                ),
            )

            #set font
            fig.update_layout(
                font = dict(size=16),
                title_font = dict(size=20),
                xaxis_title_font = dict(size=18),
                yaxis_title_font = dict(size=18),
            )

            fig.show()

    def save_result(self, param, score):

        #merge and create a dataframe
        param.update(score); data = param
        df_result = pd.DataFrame([data])

        #create results file and set header length as param to negate reading file
        if os.path.isfile(self.results_file) is True:
            df_saved_result = pd.read_csv(self.results_file)
            df_result = df_saved_result.append(df_result)

        df_result.to_csv(self.results_file, index = False)

        return

    def get_results(self):

        df = pd.read_csv(self.results_file)
        return df

In [ ]:
a = list(range(10))
print(sum(a))

In [ ]:
class ML(Base): #child

    def run_optim(self, nu_max = 11, degrees = 9, use_small_arch_list = False):

        self.model = None #clear any models if there should be one

        if use_small_arch_list is False:
            arch_list = self.__generate_arch_list_all(
                n_layers            = 4,
                n_node_steps_div    = 3,
                min_nodes           = 10
            )
        elif use_small_arch_list is True:
            arch_list = self.__generate_arch_list_limited()

        for arch in arch_list:
            print( f"Progress of optim:\t{round((arch_list.index(arch) / len(arch_list)) * 100,1)}",end = "\r")

            #refromat to save all comparison data
            param = {
                "hidden_layer_sizes"        : arch,
                "n_layers"                  : len(arch),
                "n_neurons"                 : sum(arch),
                "mean_neurosn_per_layer"    : sum(arch) / len(arch),
            }

            #create model scoring to evalute models
            score = self.create_model(arch = arch, single_model = False)
            self.save_result(param = param, score = score)

        print("Optim successfull. Read results with self.get_results()")
        return

    def __generate_arch_list_limited(self, arch_log = 2, lin_arch_scaling = 2, n_layers = 5):

        arch_list : list = []
        n_features = len(self.df_train_x.columns.tolist())

        #nodes_pow_2 = [3 ** (cone_arch_base_power + p) for p in range(1, n_layers + 1)][::-1]
        nodes_log_2 = [int(n_features * (1 / arch_log ** i)) for i in range(1,n_layers + 1)]

        for n_layer in range(1, n_layers + 1):

            #linear
            for size in  [n_features / (i * lin_arch_scaling) for i in range(1,4)]:
                arch_lin = [int(size)] * n_layer
                arch_list.append(arch_lin)

            #cone
            arch_cone = nodes_log_2[:n_layer]
            arch_list.append(arch_cone)

            #cone r
            arch_cone_r = arch_cone[::-1]
            arch_list.append(arch_cone_r)

        return arch_list

    def __generate_arch_list_all(self, n_layers = 4, n_node_steps_div = 3, min_nodes = 10):

        n_input_nodes : int = len(self.df_train_x.columns)
        arch_list = []
        n_node_list : list = [n_input_nodes]

        #set fixed params
        min_node_division = 2 #minumum number of nodes on a layer

        #divisonal
        counter = 1

        while True:

            n_nodes = int(n_input_nodes / (n_node_steps_div ** counter))

            if n_nodes < min_nodes:
                break

            n_node_list.append(n_nodes)
            counter += 1

        #clean up
        n_node_list = list(set(n_node_list)); n_node_list.append(0); n_node_list.sort()

        #create archs
        for i in range(1, (len(n_node_list)**n_layers) + 1):

            arch = []

            for j in list(range(n_layers))[::-1]:

                v =  int((i % (len(n_node_list) ** (j + 1)) / (len(n_node_list) ** j)))
                arch.append(n_node_list[v])

            arch = [k for k in arch if k != 0] #remove zero value

            if arch in arch_list:
                continue
            arch_list.append(arch)

        return arch_list

    def create_model(self, arch, single_model = True, param = None):
        """if single_model == False:
            the scores get retuned
            self.mode is not set
        elif single_model == True:
            scores do not get returned
            seld.model is set"""

        #if no parameters are given, the following default params are used
        if param is None:
            param = self.default_param

        #create model
        if self.model_metric == "r":
            ml_model = mlr
            #multi_output = MultiOutputRegressor

        elif self.model_metric == "c":
            ml_model = mlc
            #multi_output = MultiOutputClassifier

        model = ml_model(
            random_state            = self.random_state,
            hidden_layer_sizes      = arch,
            **param,
        )

        #model = multi_output(
        #    n_jobs = self.n_jobs,
        #    estimator = ml_model(
        #        random_state    = self.random_state,
        #        **param, #unpack the dict and dumps its values
        #    )
        #)

        #fit model
        model.fit(X = self.df_train_x, y = self.df_train_y)

        #set according metrics
        if single_model is True:
            self.model = model
            print(self.model)
            return

        elif single_model is False:
            score = self.get_model_score(model)
            return score

In [ ]:
mlc_obj = ML(
    df              = df,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 5,

    data_folder     = data_folder,
    results_file    = "optim_reults_mlc.csv",

    model_metric    =  "c", # r = regression, c = classification
    window          = 30,
)

In [ ]:
if run_optim is True:
    mlc_obj.run_optim()

## 5.2 Model evalution

In [ ]:
df_results = mlc_obj.get_results()
df_results.sort_values(by = "valid_accuracy_t1", ascending = False, inplace = True)

df_results.head(10)


In [ ]:
df_results.sort_values(by = "valid_accuracy_t2", ascending = False, inplace = True)
df_results.head(10)

In [ ]:
df_results.sort_values(by = "valid_accuracy", ascending = False, inplace = True)
df_results.head(10)

In [ ]:
for col in ["valid_accuracy", "valid_accuracy_t1", "valid_accuracy_t2"]:

    fig = px.scatter(
        data_frame = df_results,
        x = "n_neurons",
        y = col,
        color = "mean_neurosn_per_layer",
        #size = "n_layers",
        color_continuous_scale = plt_style_c,

        title = "Fitting graph: Multi-layer perceptron classifier",
        opacity = 1,
        trendline = "lowess",

        labels = {"mean_neurosn_per_layer": "mean neurons\nper layer"}
    )

    #fig.update_traces(marker=dict(line=dict(color='rgba(0, 0, 0, 0)')))

    scale_show(fig)

In [ ]:
class Networkplotter():

    def plot_ann(arch = [10,3,2,1], title = "ANN architecture"):

        structure = Networkplotter.create_neurons(arch) #df
        connections = Networkplotter.create_connections(structure, arch) #dict
        Networkplotter.draw_network(structure, connections, arch, title)

    def create_neurons(arch):

        structure = {
            "layer_pos"  : [],
            "neuron_pos" : [],
        }

        max_neurons = max(arch)
        mid_pos = max_neurons / 2

        for i in range(len(arch)):

            neuron_pos = mid_pos - (arch[i] / 2)

            for neuron in range(arch[i]):

                structure["layer_pos"].append(i),
                structure["neuron_pos"].append(neuron_pos)
                neuron_pos += 1

        return pd.DataFrame(structure)

    def create_connections(structure, arch):

        connections = {
            "x" :   [], #(x1,x2), (x1,x2), layer_pos
            "y" :   [], #(y1,y2), (y1,y2), neuron_pos
        }

        relevant_layers = list(range(len(arch)))[:-1]
        relevant_neurons = structure.loc[structure["layer_pos"].isin(relevant_layers)]

        for i in range(relevant_neurons.shape[0]):

            x1 = structure.iloc[i]["layer_pos"]
            y1 = structure.iloc[i]["neuron_pos"]
            x2 = x1 + 1

            for j in structure.loc[structure["layer_pos"] == x2].index.tolist():
                y2 = float(structure.iloc[j]["neuron_pos"])

                connections["x"].append((x1,x2))
                connections["y"].append((y1,y2))

        return connections

    def draw_network(structure, connections, arch, title):

        width   = len(arch) * 150
        height  = 700
        structure["size"] = 1

        fig_base = px.scatter(
            data_frame = structure,
            x = "layer_pos",
            y = "neuron_pos",
            size_max = 10,
            size = "size",

            title = title,

            width = width,
            height = height,
            #color = "neuron_pos",
            labels = {"layer_pos" : "layer", "neuron_pos" : "",}
        )

        data = fig_base.data
        for i in range(len(list(connections["x"]))):

            fig_base.add_shape(
                type='line',
                x0 = connections["x"][i][0], y0 = connections["y"][i][0],
                x1 = connections["x"][i][1], y1 = connections["y"][i][1],
                line=dict(color="lightgrey", width=2),
                layer = "below",
            )

        tick_text = list(range(len(arch)))
        tick_text[0] = "Input layer"
        tick_text[-1] = "Output layer"

        fig_base.update_layout(
            xaxis = dict(
                tickmode = 'array',
                tickvals = list(range(len(arch))),
                ticktext = tick_text,
            )
        )

        #unlcean code
        fig_base.update_yaxes(showticklabels=False)
        fig_base.update_layout(
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False)
        )
        fig_base.update_layout({
            "plot_bgcolor": "rgba(255, 255, 255, 255)",
            "paper_bgcolor": "rgba(255, 255, 255, 255)",
            })

        fig_base.show()


In [ ]:
ann_arch = [len(mlc_obj.df_train_x.columns.tolist()), 12, 12, 12, 330, 4]
ann_arch = [int(item / 4) for item in ann_arch]

print(ann_arch)


#Networkplotter.plot_ann(
#    arch = ann_arch,
#    title = "ANN architecture: MLP classifier",
#)

In [ ]:
mlc_obj.create_model(arch = [12, 12, 12, 330])

In [ ]:
optimal_param = pd.DataFrame(mlc_obj.default_param, index = ["optimal paraeters"])
optimal_param["hidden layer size"] = str([12, 12, 12, 330])
optimal_param.T

In [ ]:
scores = mlc_obj.get_model_score()

In [ ]:
mlc_obj.plot_confusion_mat(set = "valid")

In [ ]:
mlc_obj.get_model_score(get_test_score = True)

In [ ]:
mlc_obj.plot_confusion_mat(set = "test")

In [ ]:
#further improvements

#overwrite defualt params

for alpha in [1,0.5,0.25, 0.1, 0.01, 0.001]:
    print(f"\nAlpha: {alpha}")
    mlc_obj.default_param = {
        "activation"        : "relu",
        "solver"            : "adam",
        "alpha"             : alpha,         #defualt:   0.1
        "learning_rate"     : "adaptive",
        "shuffle"           : False,
        "early_stopping"    : True,
        "max_iter"          : 200,
    }

    mlc_obj.create_model(arch = [12, 12, 12, 330])
    mlc_obj.get_model_score()

## 5.3 Overfitting prevention

## 5.3 Regression model

In [ ]:
mlr_obj = ML(
    df              = df,
    y_col           = ["t2m_t1", "t2m_t2"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 5,

    data_folder     = data_folder,
    results_file    = "optim_reults_mlr.csv",

    model_metric    =  "r", # r = regression, c = classification
    window          = 30,
)

In [ ]:
if run_optim is True:
    mlr_obj.run_optim(use_small_arch_list = True)

In [ ]:
df_results = mlr_obj.get_results()
df_results.sort_values(by = "valid_rmse", ascending = True).head(5)

In [ ]:
df_results.sort_values(by = "valid_r^2", ascending = False).head(5)

In [ ]:
for arch in df_results["hidden_layer_sizes"].tolist():
    print(arch, end = ", ")

In [ ]:
mlr_obj.create_model(arch = [496, 248, 124, 62])

In [ ]:
mlr_obj.get_model_score(get_test_score = True, unstandardize_score = False)

In [ ]:
mlr_obj.get_model_score(get_test_score = True, unstandardize_score = True)

In [ ]:
draw_nn = False

if draw_nn is True:
    ann_arch = [496 * 2, 496, 248, 124, 62, 2]
    ann_arch = [int(item / 25) if int(item / 25) > 0 else 1 for item in ann_arch]

    print(ann_arch)


    Networkplotter.plot_ann(
        arch = ann_arch,
        title = "ANN architecture: MLP regressor",
    )

# 6. Recurrent neural network

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

#folders
data_folder = "data"

#machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K

#model scoring
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
run_optim = False

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_main.csv"))
df.head().T

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

In [ ]:
class Base(): #parent

    def __init__(self, df : object, y_col : list, data_folder : str, results_file : str, model_metric : str, n_jobs : int = 1, window : int = 30):

        #save raw df
        self.df_raw         = df.copy()

        #drop forbidden cols
        df = df.copy() #pass by value
        df = Base.__drop_forbidden_cols(df, y_col)

        #set dataframe for refferencing
        self.df             = df.copy() #windowed df, copy because obj is passed by refference

        #get and get x_col and y_col
        self.y_col          = y_col
        self.x_col          = list(df.drop(labels = y_col, axis = 1, inplace = False).columns.to_list())

        #misc params
        self.random_state   = 42
        self.n_jobs         = n_jobs
        self.data_folder    = data_folder
        self.results_file   = os.path.join(data_folder,results_file)
        self.model_metric   = model_metric

        #ann parameters
        self.acitvation_func        = "selu"
        self.solver                 = tf.keras.optimizers.legacy.SGD(learning_rate=0.0001) #tf.keras.optimizers.SGD(learning_rate=0.0001) #tf.keras.optimizers.Adam(learning_rate=0.001)
        self.initializer            = tf.keras.initializers.HeNormal(seed = self.random_state)
        self.n_epochs               = 5

        #windowing parameters
        self.x_window       = window #number of shifting window input features

        self.__setup()

        return

    def __setup(self):

        #order was chosen to minimize data loss, at the cost of more needed processing power

        #data preparation
        self.__windowing()
        self.__split_data()
        self.__standardize_data()

        #setup of metrics and results
        self.__set_assesment()

        return

    @staticmethod
    def __drop_forbidden_cols(df, y_col):

        forbidden_cols = ['date','t2m_t1', 't2m_t2', 't2m_t1_mean', 't2m_t2_mean', 't2m_t1_cat', 't2m_t2_cat']

        #prevent y_cols from being dropped from the data frame
        for y in y_col:
            if y in forbidden_cols:
                forbidden_cols.remove(y)

        #drop forbidden cols, to prevent adding future information to the time series
        print(f"Removed forbidden cols:\n{forbidden_cols}")
        df.drop(labels = forbidden_cols, axis = 1, inplace = True)

        return df

    def __windowing(self):
        """creates the windowed data frame"""

        self.x_col_windowed = self.x_col.copy() #copy, becaus lists are past by refference

        #input fetures: x
        for i in range(1, self.x_window + 1):
            for x_col in self.x_col: #inefficient but works just fine

                x_col_i             = f"{x_col}_-{i}"
                self.df[x_col_i]    = df[x_col].shift(i)

                self.x_col_windowed.append(x_col_i)

        #clean na columns, which were caused by the shifts
        self.df.dropna(inplace = True)
        print(f"\nApplying shifitng window:\nx_window: -{self.x_window}")

        return

    def __split_data(self):

        #reset index for splitting data
        self.df.reset_index(inplace = True, drop = True)
        length = self.df.shape[0]

        #setting split value fractions
        valid_frac_0      = 0.1
        test_frac_1       = 0.05
        train_frac_2      = 0.7
        valid_frac_3      = 0.1
        test_frac_4       = 0.05

        #get end indexes
        index_end_list = []
        cum_frac = 0

        for frac in [valid_frac_0, test_frac_1, train_frac_2, valid_frac_3, test_frac_4]:
            cum_frac += frac
            index_end_list.append(round(length * cum_frac))

        #get indexes (ugly code)
        df_indexes = self.df.index.tolist()
        train_i     = df_indexes[index_end_list[1] : index_end_list[2]]
        valid_i     = df_indexes[ : index_end_list[0]]                      + df_indexes[index_end_list[2] : index_end_list[3]]
        test_i      = df_indexes[index_end_list[0] : index_end_list[1]]     + df_indexes[index_end_list[3] : index_end_list[4]]

        #get df from indexes
        self.df_train_x = self.df[self.x_col_windowed].loc[self.df.index.isin(train_i)]
        self.df_train_y = self.df[self.y_col].loc[self.df.index.isin(train_i)]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].loc[self.df.index.isin(valid_i)]
        self.df_valid_y = self.df[self.y_col].loc[self.df.index.isin(valid_i)]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].loc[self.df.index.isin(test_i)]
        self.df_test_y = self.df[self.y_col].loc[self.df.index.isin(test_i)]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        #set data for plotting in raw df
        self.df_raw["set"] = None
        for index_items, set_type in zip([train_i, valid_i, test_i],["train", "valid", "test"]):
            self.df_raw.loc[self.df_raw.index.isin(index_items), "set"] = set_type

        return

    def plot_set_distribution(self, plotter, style, plt_style):

        if (style == "histogram") and (self.model_metric == "c"): #only plotlable with classificaiton model

            fig = px.histogram(
                data_frame = self.df_raw,
                x = "set",
                color = "t2m_t2_cat",
                histfunc = "count",

                barmode = "group",
                title = "Categorical distribution of sets",
                color_discrete_sequence = plt_style,
            )

        elif (style == "scatter"):

            fig = px.scatter(
                data_frame = self.df_raw,
                x = "date",
                y = "t2m",
                color = "set",

                title = "Trend distribution of sets",
                color_discrete_sequence = plt_style,
            )

        plotter(fig)

    def __split_data_deprecated(self):

        #df length
        length = self.df.shape[0]

        #setting split values
        valid_frac     = 0.2
        test_frac      = 0.1

        #get indexes
        train_end       = round(length * (1 - (valid_frac + test_frac)))
        valid_end       = round(length * (1 - (test_frac)))
        test_end        = round(length * (1))

        #create train df
        self.df_train_x = self.df[self.x_col_windowed].iloc[:train_end]
        self.df_train_y = self.df[self.y_col].iloc[:train_end]

        #create valid df
        self.df_valid_x = self.df[self.x_col_windowed].iloc[train_end:valid_end]
        self.df_valid_y = self.df[self.y_col].iloc[train_end:valid_end]

        #create valid df
        self.df_test_x = self.df[self.x_col_windowed].iloc[valid_end:test_end]
        self.df_test_y = self.df[self.y_col].iloc[valid_end:test_end]

        #check
        print("\nSplitting data:")
        for df, df_type in zip (
            [self.df_train_y,self.df_valid_y, self.df_test_y],
            "train,valid,test".split(",")
            ):

            print(f"{df_type} size:\t{round(df.shape[0] / length,2)}\t{df.shape[0]}")

        return

    def __standardize_data(self):

        label_cat = [0,1]; label_cat.sort()
        self.standardizing_values = {
            "x" : {},
            "y" : {},
            #    col1 : {"mean" : value, "std"  : value},
            #    col2 : {"mean" : value, "std"  : value},
            #}
            #"y" : ...
        }

        print("\nStandardizing values:")
        for col in self.df.columns:

            distinct_values = list(self.df[col].unique())
            distinct_values.sort()

            if label_cat == distinct_values: #skip categorical values
                continue

            #get mean and std for all columns across both data both data frames
            if col in self.x_col_windowed:

                self.standardizing_values["x"][col]             = {}
                self.standardizing_values["x"][col]["mean"]     = self.df_train_x[col].mean()
                self.standardizing_values["x"][col]["std"]      = self.df_train_x[col].std()

            elif col in self.y_col:

                self.standardizing_values["y"][col]             = {}
                self.standardizing_values["y"][col]["mean"]     = self.df_train_y[col].mean()
                self.standardizing_values["y"][col]["std"]      = self.df_train_y[col].std()

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = (df[col] - mean) / std #standardization

        #check sum
        print(f"Checksum train x: {self.df_train_x[list(self.standardizing_values['x'].keys())].mean().round(2).sum()}")
        print(f"Checksum train y: {self.df_train_y[list(self.standardizing_values['y'].keys())].mean().round(2).sum()}")

        return

    def unstandardize_data(self):

        #apply values
        for df, col_type in zip([self.df_train_x, self.df_valid_x, self.df_test_x, self.df_train_y, self.df_valid_y, self.df_test_y], ["x","x","x","y","y","y"]):
            for col in self.standardizing_values[col_type].keys():

                mean    = self.standardizing_values[col_type][col]["mean"]
                std     = self.standardizing_values[col_type][col]["std"]
                df[col] = df[col] * std + mean #reversed standardization

        return

    def __set_assesment(self):

        #used as a custom scoring metric for regression models
        def r_squared(y_true, y_pred):
            ss_res = K.sum(K.square(y_true - y_pred))
            ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
            return 1 - ss_res / (ss_tot + K.epsilon())

        #set loss functions and metrics based on model type

        if self.model_metric == "c":
            self.get_model_score = self.__get_model_score_c
            self.kears_metrics = [tf.keras.metrics.BinaryAccuracy()]
            self.loss_func = tf.keras.losses.BinaryCrossentropy()

        elif self.model_metric == "r":
            self.get_model_score = self.__get_model_score_r
            self.kears_metrics = [tf.keras.metrics.MeanSquaredError(), r_squared]
            self.loss_func = tf.keras.losses.MeanSquaredError()

        else:
            raise ValueError("only allowed model metrics are: r, c")

    def __get_model_score_c(self, model = None, get_test_score = False): #used, when model_metric == "c"

        #default
        get_conf_mat = False
        mat_labels = [0,1]

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            get_conf_mat = True #only get confuciton matrix when a single model is created
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        #y_train_pred    = model.evaluate(self.df_train_x, self.df_train_y)
        #y_valid_pred    = model.evaluate(self.df_valid_x, self.df_valid_y)
        #y_test_pred     = model.evaluate(self.df_test_x, self.df_test_y)

        y_train_pred    = (model.predict(self.df_train_x) > 0.5).astype(int)
        y_valid_pred    = (model.predict(self.df_valid_x) > 0.5).astype(int)
        y_test_pred     = (model.predict(self.df_test_x) > 0.5).astype(int)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get acc
            score[f"{raw_key}_accuracy_t1"]     = round(accuracy_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_accuracy_t2"]     = round(accuracy_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_accuracy"]        = round(accuracy_score(y_true = y, y_pred = y_pred),3)

            #get conf mat
            if get_conf_mat is True:
                score[f"{raw_key}_mat_t1"]     = confusion_matrix(y_true = y_t1, y_pred = y_pred_t1, labels = mat_labels)
                score[f"{raw_key}_mat_t2"]     = confusion_matrix(y_true = y_t2, y_pred = y_pred_t2, labels = mat_labels)
                #score[f"{raw_key}_mat"]        = confusion_matrix(y_true = y, y_pred = y_pred) #multi labels are not supported

        #return metrics
        if get_conf_mat is True:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def __get_model_score_r(self, model = None, get_test_score = False): #used, when model_metric == "c"
        """not yet fitted to keras"""

        set_score = False

        #if a model is passed, the function is calles from run_optim, otherwise,
        if model is None:
            model = self.model
            set_score = True
        if model is None:
            print("No model has been set. Create a model first or pass one as a param")
            return

        score = {}

        #create predictions
        y_train_pred    = model.predict(self.df_train_x)
        y_valid_pred    = model.predict(self.df_valid_x)
        y_test_pred     = model.predict(self.df_test_x)

        #seperate t1 and t2 for individual scoring
        for raw_key, y_pred, y in zip(
            ["train",           "valid",            "test"],
            [y_train_pred,      y_valid_pred,       y_test_pred],
            [self.df_train_y,   self.df_valid_y,    self.df_test_y],
        ):

            #not fetting test accurarcy if not set
            if (get_test_score == False) and raw_key == "test":
                continue

            #split
            y_pred_t1 = y_pred[:,0]
            y_pred_t2 = y_pred[:,1]

            y_t1 = y[self.y_col[0]]
            y_t2 = y[self.y_col[1]]

            #get r^2
            score[f"{raw_key}_r^2_t1"]      = round(r2_score(y_true = y_t1, y_pred = y_pred_t1),3)
            score[f"{raw_key}_r^2_t2"]      = round(r2_score(y_true = y_t2, y_pred = y_pred_t2),3)
            score[f"{raw_key}_r^2"]         = round(r2_score(y_true = y, y_pred = y_pred),3)

            #get rmse
            score[f"{raw_key}_rmse_t1"]      = round(np.sqrt(mean_squared_error(y_true = y_t1, y_pred = y_pred_t1)),3)
            score[f"{raw_key}_rmse_t2"]      = round(np.sqrt(mean_squared_error(y_true = y_t2, y_pred = y_pred_t2)),3)
            score[f"{raw_key}_rmse"]         = round(np.sqrt(mean_squared_error(y_true = y, y_pred = y_pred)),3)

        #return metrics
        if set_score:
            self.score = score
            [print(f"{key} :\t\t{score[key]}") for key in score.keys() if isinstance(score[key],float)]
            return

        return score

    def plot_confusion_mat(self, set = "valid"):
        """set = 'train', 'valid', 'test'"""

        mat_keys = [key for key in self.score.keys() if ("mat" in key) and (set in key)]

        for mat_key in mat_keys:

            mat = self.score[mat_key]
            title = str(mat_key).replace("_mat_", " ")

            fig  = px.imshow(
                mat,
                color_continuous_scale = px.colors.sequential.haline_r,
                text_auto = True,
            )

            #labels and layout
            fig.update_layout(

                title = f"Confusion matrix: {title}",

                width=500,
                height=500,

                xaxis_title="Predicted label",
                yaxis_title="True label",

                xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"]
                ),

                yaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1],
                    ticktext = ["above", "below"],
                ),
            )

            #set font
            fig.update_layout(
                font = dict(size=16),
                title_font = dict(size=20),
                xaxis_title_font = dict(size=18),
                yaxis_title_font = dict(size=18),
            )

            fig.show()

    def save_result(self, param, score):

        #merge and create a dataframe
        param.update(score); data = param
        df_result = pd.DataFrame([data])

        #create results file and set header length as param to negate reading file
        if os.path.isfile(self.results_file) is True:
            df_saved_result = pd.read_csv(self.results_file)
            df_result = df_saved_result.append(df_result)

        df_result.to_csv(self.results_file, index = False)

        return

    def get_results(self):

        df = pd.read_csv(self.results_file)
        return df

In [ ]:
class EarlyStopping(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):

      if(logs.get("accuracy") < 0.001):
          print("\nMAEthreshold reached. Training stopped.")
          self.model.stop_training = True

In [ ]:
class RN(Base):

    def run_optim(self):

        self.model = None #clear any models if there should be one

        arch_list = self.__generate_arch_list(
            arch_log               = 2,
            lin_arch_scaling       = 2,
            n_layers               = 5,
        )

        for arch in arch_list:
            print( f"Progress of optim:\t{round((arch_list.index(arch) / len(arch_list)) * 100,1)}",end = "\r")

            #refromat to save all comparison data
            if arch[0] == arch[-1]:
                shape = "linear"
            elif arch[0] > arch[-1]:
                shape = "cone"
            elif arch[0] < arch[-1]:
                shape = "cone_r"

            param = {
                "hidden_layer_sizes"        : str(arch),
                "n_layers"                  : len(arch),
                "n_neurons"                 : sum(arch),
                "shape"                     : shape,
            }

            #create model scoring to evalute models
            score = self.create_model(arch = arch, single_model = False)
            self.save_result(param = param, score = score)

        print("Optim successfull. Read results with self.get_results()")
        return

    def __generate_arch_list(self, arch_log = 2, lin_arch_scaling = 2, n_layers = 5):

        arch_list : list = []
        n_features = len(self.df_train_x.columns.tolist())

        #nodes_pow_2 = [3 ** (cone_arch_base_power + p) for p in range(1, n_layers + 1)][::-1]
        nodes_log_2 = [int(n_features * (1 / arch_log ** i)) for i in range(1,n_layers + 1)]

        for n_layer in range(1, n_layers + 1):

            #linear
            for size in  [n_features / (i * lin_arch_scaling) for i in range(1,4)]:
                arch_lin = [int(size)] * n_layer
                arch_list.append(arch_lin)

            #cone
            arch_cone = nodes_log_2[:n_layer]
            arch_list.append(arch_cone)

            #cone r
            arch_cone_r = arch_cone[::-1]
            arch_list.append(arch_cone_r)

        return arch_list

    def create_model(self, arch, single_model = True):

        shape = [len(self.df_train_x.columns.tolist())]

        #init model
        model = tf.keras.models.Sequential()

        #dynamicaly scaling input layer
        model.add(tf.keras.layers.Lambda(
            lambda x: tf.expand_dims(x, axis=-1),
            input_shape=shape, #working: [None]
        ))
        """

        model.add(tf.keras.layers.Dense(
            units = len(self.df_train_x.columns.tolist()),
            input_dim = 1,
            activation = self.acitvation_func,
        ))
        """

        #add LSTM layers as hidden layers
        for i in range(len(arch)):

            if (i + 1) == len(arch):
                return_sequences = False
            else:
                return_sequences = True

            #hidden layers
            model.add(tf.keras.layers.Bidirectional(
                tf.keras.layers.LSTM(
                    arch[i],
                    #kernel_initializer = self.initializer,
                    activation = self.acitvation_func,
                    return_sequences = return_sequences,
                )
            ))

        #add a Dense output layer
        model.add(tf.keras.layers.Dense(units = 2, activation = self.acitvation_func)) #untis = 2, two binary classes

        model = self.__compile_model(model)

        #set according metrics
        if single_model is True:
            self.model = model
            print(self.model)
            return None

        elif single_model is False:
            score = self.get_model_score(model)
            return score

    def __compile_model(self, model):

        model.compile(
            loss = self.loss_func,
            optimizer = self.solver,
            metrics =  self.kears_metrics #accuracy
        )

        model.fit(
            x = self.df_train_x,
            y = self.df_train_y,
            #validation_data = (self.df_valid_x, self.df_valid_y),
            shuffle = False, #keep in order because it is time series data
            epochs = self.n_epochs,
            callbacks =[EarlyStopping()]
        )

        return model

In [ ]:
rnc_obj = RN(
    df              = df,
    y_col           = ["t2m_t1_cat", "t2m_t2_cat"], #or ["t2m_t1_cat", "t2m_t1_cat"]

    n_jobs          = 5, #not applicable for keras models

    data_folder     = data_folder,
    results_file    = "optim_reults_rnc.csv",

    model_metric    =  "c", # r = regression, c = classification
    window          = 30,
)

In [ ]:
if run_optim:
    rnc_obj.run_optim()

In [ ]:
rnc_obj.get_results()

In [ ]:
class Networkplotter():

    def plot_ann(arch = [10,3,2,1], title = "ANN architecture"):

        structure = Networkplotter.create_neurons(arch) #df
        connections = Networkplotter.create_connections(structure, arch) #dict
        Networkplotter.draw_network(structure, connections, arch, title)

    def create_neurons(arch):

        structure = {
            "layer_pos"  : [],
            "neuron_pos" : [],
        }

        max_neurons = max(arch)
        mid_pos = max_neurons / 2

        for i in range(len(arch)):

            neuron_pos = mid_pos - (arch[i] / 2)

            for neuron in range(arch[i]):

                structure["layer_pos"].append(i),
                structure["neuron_pos"].append(neuron_pos)
                neuron_pos += 1

        return pd.DataFrame(structure)

    def create_connections(structure, arch):

        connections = {
            "x" :   [], #(x1,x2), (x1,x2), layer_pos
            "y" :   [], #(y1,y2), (y1,y2), neuron_pos
        }

        relevant_layers = list(range(len(arch)))[:-1]
        relevant_neurons = structure.loc[structure["layer_pos"].isin(relevant_layers)]

        for i in range(relevant_neurons.shape[0]):

            x1 = structure.iloc[i]["layer_pos"]
            y1 = structure.iloc[i]["neuron_pos"]
            x2 = x1 + 1

            for j in structure.loc[structure["layer_pos"] == x2].index.tolist():
                y2 = float(structure.iloc[j]["neuron_pos"])

                connections["x"].append((x1,x2))
                connections["y"].append((y1,y2))

        return connections

    def draw_network(structure, connections, arch, title):

        width   = len(arch) * 150
        height  = 700
        structure["size"] = 1

        fig_base = px.scatter(
            data_frame = structure,
            x = "layer_pos",
            y = "neuron_pos",
            size_max = 10,
            size = "size",

            title = title,

            width = width,
            height = height,
            #color = "neuron_pos",
            labels = {"layer_pos" : "layer", "neuron_pos" : "",}
        )

        data = fig_base.data
        for i in range(len(list(connections["x"]))):

            fig_base.add_shape(
                type='line',
                x0 = connections["x"][i][0], y0 = connections["y"][i][0],
                x1 = connections["x"][i][1], y1 = connections["y"][i][1],
                line=dict(color="lightgrey", width=2),
                layer = "below",
            )

        tick_text = list(range(len(arch)))
        tick_text[0] = "Input layer"
        tick_text[-1] = "Output layer"

        fig_base.update_layout(
            xaxis = dict(
                tickmode = 'array',
                tickvals = list(range(len(arch))),
                ticktext = tick_text,
            )
        )

        #unlcean code
        fig_base.update_yaxes(showticklabels=False)
        fig_base.update_layout(
            xaxis=dict(showgrid=False),
            yaxis=dict(showgrid=False)
        )
        fig_base.update_layout({
            "plot_bgcolor": "rgba(255, 255, 255, 255)",
            "paper_bgcolor": "rgba(255, 255, 255, 255)",
            })

        fig_base.show()


In [ ]:
ann_arch = [len(rnc_obj.df_train_x.columns.tolist()), 496, 496, 496, 4]
ann_arch = [int(item / 30) if int(item / 30) > 0 else 1 for item in ann_arch]

print(ann_arch)

Networkplotter.plot_ann(
    arch = ann_arch,
    title = "ANN architecture: RNN classifier",
)

# 7. Gas price and temperature correlations

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from datetime import timedelta

#folders
data_folder = "data"

#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    fig.update_layout(
        width=1500,
        height=750,
    )

    #show
    fig.show()

    return

## 7.1 Data understanding (gathering, cleaning)
- source: https://www.eia.gov/dnav/ng/hist/rngwhhdD.htm

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_main.csv"))
df = df[["date", "t2m", "t2m_t1_cat"]]
df.head()

In [ ]:
df_gas_raw = pd.read_csv(os.path.join(data_folder, "raw_gas", "raw_download.csv"))
df_gas_raw.tail()

In [ ]:
df_gas = df_gas_raw
df_gas.isna().sum()

In [ ]:
df_gas["gas_usd_spot"].interpolate(inplace = True, method = "linear")

In [ ]:
df_gas = df_gas_raw
df_gas.isna().sum()

In [ ]:
fig = px.line(
    data_frame= df_gas,
    x = "date",
    y = "gas_usd_spot",
    title = "Natural gas price, spot (USD per Million Btu)",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
df = pd.merge(right=df, left = df_gas, on = "date", how = "inner")
df

## 7.2 Data preparation

In [ ]:
#calculate percentual change
df["t2m_change"] = (((df["t2m"] - df["t2m"].shift(1)) / df["t2m"].shift(1)) * 100).round(2)
df["gas_usd_spot_change"] = (((df["gas_usd_spot"] - df["gas_usd_spot"].shift(1)) / df["gas_usd_spot"].shift(1)) * 100).round(2)
df["date"] = pd.to_datetime(df["date"], format = "%Y-%m-%d")

df.dropna(inplace = True)
df

In [ ]:
#standardize values to make them compareable
for col in ["gas_usd_spot", "t2m", "t2m_change", "gas_usd_spot_change"]:
    df[col] = (df[col] - df[col].mean()) / df[col].std()

In [ ]:
fig = px.scatter(
    data_frame = df,
    x = "t2m_change",
    y = "gas_usd_spot_change",
    color_discrete_sequence = plt_style_s,
    title = "Gas price change and temperature change comparison",
    trendline = "ols",
    trendline_color_override = "red",
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df,
    x = "t2m",
    y = "gas_usd_spot",
    color_discrete_sequence = plt_style_s,
    title = "Gas price change and temperature change comparison",
    trendline = "ols",
    trendline_color_override = "red",
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df.loc[df["date"].dt.month.isin([10,11,12,1,2,3])],
    x = "t2m",
    y = "gas_usd_spot",
    color_discrete_sequence= plt_style_s,
    title = "Gas price change and temperature change comparison",
    trendline = "ols",
    trendline_color_override = "red",
    facet_col = "t2m_t1_cat",
)

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df.loc[df["date"].dt.month.isin([10,11,12,1,2,3])],
    x = "t2m_change",
    y = "gas_usd_spot_change",
    color_discrete_sequence = plt_style_s,
    title = "Gas price change and temperature change comparison",
    trendline = "ols",
    trendline_color_override = "red",
    facet_col = "t2m_t1_cat",
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df.iloc[-200:],
    x = "date",
    y = ["t2m_change", "gas_usd_spot_change"],
    title = "Change over time",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df.iloc[-40000:],
    x = "date",
    y = ["t2m", "gas_usd_spot"],
    title = "Stanardized values over time",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
#apply z score peaks
#source: https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/43512887#43512887

def thresholding_algo(y, lag, threshold, influence):
    """Robust peak detection algorithm (using z-scores)

    Args:
        y (_type_): y_vector / time series
        lag (_type_): the lag of the moving window
        threshold (_type_): the z-score at which the algorithm signals
        influence (_type_): the influence (between 0 and 1) of new signals on the mean and

    Returns:
        _type_: dict {
            singals
            avgFilter
            stdFilter
        }
    """

    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)

    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])

    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
#get score
for col in ["t2m", "gas_usd_spot"]:


    df[f"{col}_zscore"] = thresholding_algo(y = df[col], lag = 360, threshold = 2.5, influence = 0.1)["signals"]

    #get peak spots for plotting
    df[f"{col}_zscore"] = (df[col] * df[f"{col}_zscore"]) * df[f"{col}_zscore"]

    #set others to NaN to for plotting as scatter
    df.loc[df[f"{col}_zscore"] == 0, f"{col}_zscore"] = None


In [ ]:
fig = px.line(
    data_frame = df.iloc[-10000:],
    x = "date",
    y = "t2m",
    title = "Stanardized values over time",
    color_discrete_sequence = plt_style_s,
)

fig.add_scatter(
    x = df["date"],
    y = df["t2m_zscore"],
    mode = "markers",
    name = "local z score peaks t2m",
    marker = {"size" : 10},

)

scale_show(fig)

In [ ]:
def fixed_lower_th(y, th = -2.4):

    signal = (y <= th).astype(int)
    signal = signal * -1

    return signal

In [ ]:
#get score
for col in ["t2m", "gas_usd_spot"]:

    df[f"{col}_th"] = fixed_lower_th(y = df[col], th = -2.4)

    #get peak spots for plotting
    df[f"{col}_th"] = (df[col] * df[f"{col}_th"]) * df[f"{col}_th"]

    #set others to NaN to for plotting as scatter
    df.loc[df[f"{col}_th"] == 0, f"{col}_th"] = None

In [ ]:
fig = px.line(
    data_frame = df.iloc[-10000:],
    x = "date",
    y = "t2m",
    title = "Stanardized values over time",
    color_discrete_sequence = plt_style_s,
)

fig.add_scatter(
    x = df["date"],
    y = df["t2m_th"],
    mode = "markers",
    name = "local z score peaks t2m",
    marker = {"size" : 10},

)

scale_show(fig)

In [ ]:
df["t2m_zscore"] = thresholding_algo(y = df["t2m"], lag = 60, threshold = 3.5, influence = 0.5)["signals"]

In [ ]:
df.loc[df["t2m_th"].isna() == False, "t2m_th"] = 10
df.loc[df["t2m_th"].isna() == True, "t2m_th"] = -10

df["t2m_zscore"] = df["t2m_zscore"].abs() * 20 - 10

In [ ]:
fig = px.line(
    data_frame = df,
    x = "date",
    y = ["gas_usd_spot", "t2m_th", "t2m_zscore"],
    title = "Gas price",
    color_discrete_sequence = plt_style_s,

)

fig.update_layout(yaxis_range=[-3.5, 10])

scale_show(fig)

In [ ]:
fig = px.line(
    data_frame = df,
    x = "date",
    y = ["gas_usd_spot", "t2m_zscore",],
    title = "Gas price",
    color_discrete_sequence = plt_style_s,

)

fig.update_layout(yaxis_range=[-3.5, 10])

scale_show(fig)

In [ ]:
fig = px.scatter(
    data_frame = df,
    x = "date",
    y = "gas_usd_spot",
    color = "t2m_t1_cat",
    title = "Gas price",
    color_continuous_scale = plt_style_s,
)

scale_show(fig)

# 8 PV image data clustering
- process: https://towardsdatascience.com/a-step-by-step-guide-for-clustering-images-4b45f9906128
- clustering methods: https://towardsdatascience.com/from-data-to-clusters-when-is-your-clustering-good-enough-5895440a978a
- hog method for feature method: https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import random
import matplotlib.pyplot as plt
import plotly.io as pio

#image clustering library
from clustimage import Clustimage

#machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#folders
data_folder = "data"

#warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
random.seed(42)

In [ ]:
#set if data should be safed
save_data = False

In [ ]:
#plot styles
plt_style_c = px.colors.sequential.haline #complex
plt_style_s = px.colors.diverging.Portland #simple

#defualt plot size
size = {
    "width" : 1500 ,
    "height" : 750 ,
}

#function for plotting
def scale_show(fig, size_override = False):

    #set font
    fig.update_layout(
        font = dict(size=16),
        title_font = dict(size=20),
        xaxis_title_font = dict(size=18),
        yaxis_title_font = dict(size=18),
    )

    #set size
    if size_override == False:
        fig.update_layout(
            width=1500,
            height=750,
        )

    #show
    fig.show()

    return

## 8.1 data preparation

In [ ]:
df = pd.read_csv(os.path.join(data_folder, "df_pv_clustering.csv"))
#df["date"] = pd.to_datetime(df["date"])
df.drop(labels = "longitude", axis = 1, inplace = True)
df.head()

In [ ]:
df.sort_values(by = ["date", "level", "latitude"], ascending = [True, True, True], inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

In [ ]:
n_dates     = df["date"].unique().shape[0]
n_lats      = df["latitude"].unique().shape[0]
n_levels    = df["level"].unique().shape[0]

print(n_dates)
print(n_lats)
print(n_levels)


In [ ]:
#cost: high

"""
metric = "speed" #["speed", "t"]
images_na = {}

for date in df["date"].unique().tolist()[:3]:
    im = []

    for level in df["level"].unique():
        metrics = df.loc[(df["date"] == date) & (df["level"] == level)]["speed"].tolist()
        im.append(metrics)

    print(f"Compiling image: {date}", end = "\r")
    images_na[date] = im
"""

In [ ]:
def image_compiler(df, metric):

    n_lats      = df["latitude"].unique().shape[0]
    n_levels    = df["level"].unique().shape[0]

    pixel_row = []
    image = []
    images = []

    #standardize pixels
    pixels = (df[metric] - df[metric].mean()) / df[metric].std()
    pixels = pixels.tolist()

    for pixel in pixels:

        pixel_row.append(pixel)

        if len(pixel_row) == n_lats:
            image.append(np.array(pixel_row.copy()))
            pixel_row.clear()

            if len(image) == n_levels:
                images.append(np.array(image.copy()))
                image.clear()

    images = np.stack(images)

    return images

In [ ]:
images = image_compiler(df = df, metric = "speed")

In [ ]:
#checksum
print(df.shape[0] / (n_lats * n_levels))
print(len(images))

In [ ]:
px.imshow(images[0], title = "Wind speed standardized", color_continuous_scale = plt_style_s)


In [ ]:
px.imshow(images[-1], title = "Wind speed standardized", color_continuous_scale = plt_style_s)

In [ ]:
images[0]

In [ ]:
#split the data, no validiotn and test sset

def splitter (images):

    images = images.copy() #prevent altering the original list

    train_split     = 0.8
    test_split      = 0.2  #only used for idication

    index_list = list(range(len(images)))
    train_size = int(len(images) * train_split)

    train_set_i     = random.sample(population = index_list, k = train_size)
    test_set_i      = [index for index in index_list if index not in train_set_i]

    train_set       = [images[i] for i in train_set_i]
    test_set        = [images[i] for i in test_set_i]

    train_set = np.stack(train_set)
    test_set = np.stack(test_set)

    print(f"train set:\t{round(len(train_set) / len(images), 2)}\ntest set:\t{round(len(test_set) / len(images),2)}")

    return train_set, test_set

In [ ]:
train_set, test_set = splitter(images)

In [ ]:
train_set[0].shape

## 8.2 modelling (clustimage)

In [ ]:
cl = Clustimage(
    method='hog',
    embedding='tsne',
    grayscale=False,
    dim=(10,45),

    params_hog = {
        "orientations"      : 8,
        "pixels_per_cell"   : (4,4),
    },

    verbose = True,
)

In [ ]:
try:
    results = cl.fit_transform(
        train_set,
        cluster='agglomerative',
        evaluate='silhouette',
        metric='euclidean',
        linkage='ward',
        min_clust=3,
        max_clust=15,
        cluster_space='high',
    )
except :
    print("Not able to conver the datatypes of numpy arrays to image data. Twat")

In [ ]:
try:
    #create model
    cl = Clustimage(method='hog')

    #extract features
    train_set_feat = cl.extract_feat(train_set)

    # Embedding using tSNE
    xycoord = cl.embedding(train_set_feat)

    # Cluster with all default settings
    labels = cl.cluster(
        cluster='agglomerative',
        evaluate='silhouette',
        metric='euclidean',
        linkage='ward',
        min_clust=3,
        max_clust=15,
        cluster_space='high',
    )

    # Return
    results = cl.results

except:
    print("Not able to conver the datatypes of numpy arrays to image data. Twat")

## 8.3 modelling (skealrn)
- https://medium.com/@chengweizhang2012/how-to-do-unsupervised-clustering-with-keras-9e1284448437

In [ ]:
class KMC():

    def __init__ (self, n_cluster, train_set, test_set):

        #set unflattended valus
        self.train_set      = train_set.copy()
        self.test_set       = test_set.copy()

        #model params
        self.random_state   = 42
        self.n_cluster      = n_cluster
        self.algorithm      = "full"

        #flatten values
        self.train_set_flat     = self.reshape(self.train_set)
        self.test_set_flat      = self.reshape(self.test_set)

    def reshape(self, set):

        n_samples, height, width = set.shape
        images_flat = set.reshape((n_samples, height * width))

        return images_flat

    def create_model(self):

        self.model = KMeans(
            n_clusters      = self.n_cluster,
            random_state    = self.random_state,
            algorithm       = self.algorithm,
        )

        self.model.fit(self.train_set_flat)
        self.__predict_match()

        return

    def __predict_match(self):

        #match image and labels for analysis (train)
        labels_train = self.model.labels_
        self.labels = []

        for i in range(len(labels_train)):

            data = {
                "set"   : "train",
                "label" : labels_train[i],
                "im"    : self.train_set[i],
            }
            self.labels.append(data)

        #match image and labels for analysis (test)
        labels_test = self.model.predict(self.test_set_flat)

        for i in range(len(labels_test)):

            data = {
                "set"   : "test",
                "label" : labels_test[i],
                "im"    : self.test_set[i],
            }
            self.labels.append(data)

        return

In [ ]:
kmc = KMC(n_cluster = 9, train_set = train_set, test_set = test_set)
kmc.create_model()

In [ ]:
df_cluster = pd.DataFrame(kmc.labels)
df_cluster

In [ ]:
fig = px.histogram(
    data_frame = df_cluster,
    x = "label",
    histfunc = "count",
    histnorm = "probability density",
    color = "set",
    barmode = "group",

    title = "KMeans Clusters",
    color_discrete_sequence = plt_style_s,
)

scale_show(fig)

In [ ]:
kmc.labels

In [ ]:
#get a sample from each cluster to get an idea of the distribution
label_indexes = {}
sample_plots = 3

for label in df_cluster["label"].unique().tolist():
    indexes = df_cluster.loc[df_cluster["label"] == label].index.tolist()
    label_indexes[label] = indexes


#get first elemt of each cluster and plot image
keys = list(label_indexes.keys())
keys.sort()

for label in keys:

    for i in range(sample_plots):

        im_ind_all = label_indexes[label]
        im_ind = random.sample(population = im_ind_all, k = 1, )[0]

        #retrvie data for plotting
        im_data     = kmc.labels[im_ind]["im"]
        im_label    = kmc.labels[im_ind]["label"]
        im_set      = kmc.labels[im_ind]["set"]

        #generate image plot
        fig = px.imshow(
            im_data,
            title = f"Wind speed (norm) - Label: {im_label} ({im_set})",
            color_continuous_scale = plt_style_s,
            range_color = [-2,6],
            width = 1500,
            height = 500,
            )

        fig.update_xaxes(title_text="Latitude (offset by -44)")
        fig.update_yaxes(title_text="Pressure level [hPa]")

        #image_file = "test.png"
        #pio.write_image(fig, image_file, engine="plotly.io")

        #fig.write_image("test.png") #f"Wind_speed_(norm)-Label:{im_label}({im_set}).png"

        scale_show(fig, size_override = True)

In [ ]:
#get predction son the test set
# Flatten the images
n_samples, height, width = images.shape
images_flat = images.reshape((n_samples, height * width))